
Questions we would like the data to answer:

    1.0. For each industry, what is the breakdown of gender from 2009 to 2018?
    
    1.1. For each job category, what is the breakdown of gender from 2009 to 2018?
    
    2.0. For each industry, what is the breakdown of race from 2009 to 2018?
    
    2.1. For each job category, what is the breakdown of race from 2009 to 2018?
    
    3.0. What is the year to year trend for women in the Professional and Technician job categories from 2009 to 2018?
    
    3.1. What is the year to year trend for minorities in the Professional and Technician job categories from 2009 to 2018?
    
    4.0. How does the number of women compare to the number of men in the in the Professional and Technician job categories for years of 2009 to 2018?
    
    4.1. How does the number of Asian men in the Professional and Technician job categories compare to the number White men in the same categories from 2009 to 2018?
    
    Note: Tables with many categories, the percentages don't sum to 100 due to rounding.
    

In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.linalg as la
import math
import matplotlib
import matplotlib.pyplot as plt

naics = pd.read_csv('NAICS3 Industry Code Description.txt', sep = ':')
naics.columns = ['Sector', 'Description']
naics['Sector'] = naics['Sector'].str.replace('Subsector', '')
naics = naics[['Description', 'Sector']]

des_to_sect = naics.set_index(['Description']).to_dict()['Sector']


# Read in "NAICS3" text for Industry code.
# Rename column header for string matching.


naics_dict = {}

naics_dict['Crop Production'] = str(111)
naics_dict['Animal Production'] = str(112)
naics_dict['Sporting Goods, Hobby, Book, and Music Stores'] = str(451)
naics_dict['Data Processing, Hosting and Related Services'] = str(518)


#Created a dictionary using the industry description and corresponding 3 digit code as the key-value entries.
#Added missing industry categories.


for k, v in des_to_sect.items():
    k = k.strip()
    v = v.strip()
    naics_dict[k] = v
    
naics_dict = {k: v for k, v in sorted(naics_dict.items(), key = lambda item: item[1])}


# Cleaning up industry descriptors of white space then sorting from numerically smallest to largest using value entries of the NAICS3 code in the dictionary.


display(naics_dict)

## YEAR 18

In [ ]:
with open('year18_state_nac3.txt') as csvfile:
    data18 = csv.reader(csvfile, delimiter = ';')
    df18 = pd.DataFrame(data18)
    
data_dict18 = pd.read_excel(io = 'data_dictionary-2018.xls', sheet_name = 'state_NAIC3_2018') # Reads in the data from the specified spreadsheet.
data_dict18 = data_dict18.drop(data_dict18.columns[[0, 3, 4]], axis = 1) # Dropping certain columns.
data_dict18 = data_dict18.iloc[3:144] # Reframing the data.
data_dict18 = data_dict18.reset_index(drop = True) # Dropping the default index column.
data_dict18.columns = ['Variable', 'Label']
data_dict18['Job Category'], data_dict18['Race'], data_dict18['Gender'] = data_dict18['Label'].replace('[0-9.-]', '').str.split('/', 2).str
data_dict18 = data_dict18.drop('Label', axis = 1)

header18 = df18.iloc[0] # Sets the first row to "header18" variable.
df18 = df18.iloc[1:] # Returns a dataframe starting from the second row.
df18.columns = header18 # Sets the header for the returned dataframe using the first row labels in "header18".
df18 = df18.set_index(['STATE_LABEL', 'NAC3']) # Sets multilevel index based on state name and the 3 digit NAIC industry code.
df18 = df18.replace('', np.nan) # Populates empty cells with NAN.

gen_list = ['MALE', 'FEMALE']
race_list = sorted(['WHITE', 'HISPANIC', 'BLACK', 'ASIAN', 'AM-INDIAN-ALK', 'HAWAIIAN', 'TWO OR MORE RACES']) # Using the EEO-1 documentation to define races.

df18_err_gen = data_dict18[~data_dict18['Gender'].isin(gen_list)] # Finding entries under 'Gender' column that isn't male or female.
df18_errcor_gen = df18_err_gen.index.to_list() # Storing the findings in a list for examination and correction. Errors are most likely due to misspelling, capitlization, or punctuation.

df18_err_race = data_dict18[~data_dict18['Race'].isin(race_list)] # Finding entries that don't match entries under the 'Race' column.
df18_errcor_race = df18_err_race.index.to_list() # Storing the found 'Race' entries for examination and correction.

df18_err_race_hwn = df18_err_race[df18_err_race['Race'].str.upper().str.contains('HAWAI')]
df18_errcor_race_hwn = df18_err_race_hwn.index.to_list()

df18_err_race_asn = df18_err_race[df18_err_race['Race'].str.upper().str.contains('ASIA')]
df18_errcor_race_asn = df18_err_race_asn.index.to_list()

df18_err_race_lat = df18_err_race[df18_err_race['Race'].str.upper().str.contains('HISP')]
df18_errcor_race_lat = df18_err_race_lat.index.to_list()

df18_err_race_tmr = df18_err_race[df18_err_race['Race'].str.upper().str.contains('TWO')]
df18_errcor_race_tmr = df18_err_race_tmr.index.to_list()

data_dict18.loc[51, 'Gender'] = 'FEMALE'
data_dict18.loc[61, 'Gender'] = 'MALE'
data_dict18.loc[71, 'Gender'] = 'FEMALE'

data_dict18.loc[df18_errcor_race_hwn, 'Race'] = 'HAWAIIAN'
data_dict18.loc[df18_errcor_race_asn, 'Race'] = 'ASIAN'
data_dict18.loc[df18_errcor_race_lat, 'Race'] = 'HISPANIC'
data_dict18.loc[df18_errcor_race_tmr, 'Race'] = 'TWO OR MORE RACES'

#display(df18_errcor_race_tmr)
#display(df18_err_race_tmr)
display_side_by_side([data_dict18, df18],['2018 Data Dictionary','df18'])
#display(data_dict18.style.set_caption('2018 Data Dictionary'))
#display(df18)

In [ ]:
'''
with open('year18_state_nac3.txt') as csvfile:
    data18 = csv.reader(csvfile, delimiter = ';')
    df18 = pd.DataFrame(data18)
    
data_dict18 = pd.read_excel(io = 'data_dictionary-2018.xls', sheet_name = 'state_NAIC3_2018')
data_dict18 = data_dict18.drop(data_dict18.columns[[0, 3, 4]], axis = 1)
data_dict18 = data_dict18.iloc[3:144]
data_dict18 = data_dict18.reset_index(drop = True)
data_dict18.columns = ['Variable', 'Label']
data_dict18['Job Category'], data_dict18['Race'], data_dict18['Gender'] = data_dict18['Label'].replace('[0-9.-]', '').str.split('/', 2).str
data_dict18 = data_dict18.drop('Label', axis = 1)

header18 = df18.iloc[0] # Sets the first row to "header18" variable.
df18 = df18.iloc[1:] # Returns a dataframe starting from the second row.
df18.columns = header18 # Sets the header for the returned dataframe using the first row labels in "header18".
df18 = df18.set_index(['STATE_LABEL', 'NAC3']) # Sets multilevel index based on state name and the 3 digit NAIC industry code.
df18 = df18.replace([None], np.nan)

# Using the .replace() method is straightforward since the inconsis for the two races are consistent.
data_dict18['Race'] = data_dict18['Race'].replace('Hawaiia', 'HAWAIIAN')
data_dict18['Race'] = data_dict18['Race'].replace(dict.fromkeys(['Two Races', 'Two More Races'], 'TWO OR MORE RACES'))

# What if the errors from the data dictionary for race categories aren't consistent? I use .str.contains() method to search
# "Race" column for a partial match on a race.
lat_df18 = data_dict18[data_dict18['Race'].str.contains('HISPANIC', na = False)]
asn_df18 = data_dict18[data_dict18['Race'].str.contains('ASIAN|Asian', na = False)]

# I return a dataframe from a slice where the conditional statement only gives entries where there are mismatches from the preferred variable.
asn_errcor18 = asn_df18[(asn_df18['Race'] != 'ASIAN')].copy()
lat_errcor18 = lat_df18[(lat_df18['Race'] != 'HISPANIC')].copy()

# Correcting the dataframe for each race category. This method is useful if there are many entries with the same mistake.
asn_errcor18['Race'] = 'ASIAN'
lat_errcor18['Race'] = 'HISPANIC'

lat_errcor18.loc[51, 'Gender'] = 'FEMALE'

# Updating the data dictionary to be consistent with the preferred variable names for race using original dataframe indices to match.
# Instead of updating each one by row and column, I update the original dataframe with a dataframe comprised of corrected entries.
data_dict18.update(asn_errcor18, overwrite = True)
data_dict18.update(lat_errcor18, overwrite = True)

gen_errcor18 = data_dict18[(data_dict18['Gender'] != 'MALE') & (data_dict18['Gender'] != 'FEMALE') & (data_dict18['Gender'].notnull()) ]
gen_errcor18 = gen_errcor18[~(gen_errcor18['Gender'].str.contains('MINORITY') | gen_errcor18['Gender'].str.contains('PERCENT'))]

data_dict18.loc[61, 'Gender'] = 'MALE'
data_dict18.loc[71, 'Gender'] = 'FEMALE'


#display_side_by_side([asn_df18, lat_df18], ['Dataframe Only Asian Race', 'Dataframe Only Hispanic Race'])
#display(gen_errcor18.style.set_caption('Gender Correction'))
#display_side_by_side([asn_errcor18, lat_errcor18], ['Dataframe Error Correction Asian','Dataframe Error Correction Hispanic'])
display(data_dict18.style.set_caption('2018 Data Dictionary'))
display(df18)
'''

### Question 1.0. 

In [ ]:
data_dict18_male = data_dict18[(data_dict18['Gender'] == 'MALE')]
data_dict18_female = data_dict18[(data_dict18['Gender'] == 'FEMALE')]

#display_side_by_side([data_dict18_male, data_dict18_female], ['Variables by Men', 'Variables by Women'] )

men18 = data_dict18_male['Variable'].to_list()
women18 = data_dict18_female['Variable'].to_list()

df18_ind_men = df18[men18]
df18_ind_women = df18[women18]

df18_ind_men = df18_ind_men.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df18_ind_men["Men's Count"] = df18_ind_men.sum(axis = 1)

df18_ind_women = df18_ind_women.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df18_ind_women["Women's Count"] = df18_ind_women.sum(axis = 1)

#display(df18_ind_men.style.set_caption('df18_ind_men'), df18_ind_women.style.set_caption('df18_ind_women'))

q1_0_18 = pd.DataFrame(index = df18.index)
q1_0_18["Male Count"] = df18_ind_men["Men's Count"]
q1_0_18["Female Count"] = df18_ind_women["Women's Count"]
q1_0_18['Total Count'] = q1_0_18.sum(axis = 1)
q1_0_18['% Men'] = (q1_0_18['Male Count']/q1_0_18['Total Count'])
q1_0_18['% Women'] = (q1_0_18['Female Count']/q1_0_18['Total Count'])
q1_0_18['NAC3 Code Description'] = df18['NAC3_LABEL']
col_head18 = ['NAC3 Code Description', 'Male Count', 'Female Count', 'Total Count', '% Men', '% Women']
q1_0_18 = q1_0_18[col_head18]

#display(len(q1_0_18.groupby(level = 0)))
display(q1_0_18.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))



### Question 1.1. 

In [ ]:
# Returns the 10 job categories in the dataset.
job_cat18_list = data_dict18.loc[1:10, 'Job Category'].to_list()

# Corrects the slight mismatch of job categories.
for i, j in data_dict18.iterrows():
    for k in job_cat18_list:
        if j[1][:2] == k[:2]:
            data_dict18.loc[i, 'Job Category'] = k
            
# List to store the variables corresponding to job category for each race.
job_cat18_male_var = []    
job_cat18_female_var = []    

# Extracting the "variable" element from male data dictionary based on the job category matching across alll races
# then adding to a list.
for k in job_cat18_list:
    for i, j in data_dict18.iterrows():
        if (k == j[1]) and (j[3] == 'MALE'):
            job_cat18_male_var.append(j[0])

for k in job_cat18_list:
    for i, j in data_dict18.iterrows():
        if (k == j[1]) and (j[3] == 'FEMALE'):
            job_cat18_female_var.append(j[0])            
            
# Splits the entire list into 10 arrays of size 7 grouping together the job category across all races.
job_cat18_male_varsplit = np.array_split(job_cat18_male_var, 10)
job_cat18_female_varsplit = np.array_split(job_cat18_female_var, 10)

# Initialize a dictionary where I store all the generated dataframes based on the job category for all races.
job_cat18_male_df_dict = {}
job_cat18_female_df_dict = {}

# Fills the dictionary and replaces all empty cells within each dataframe with an NaN value.
n = 0
for entry in job_cat18_list:
    job_cat18_male_df_dict[entry] = df18[list(job_cat18_male_varsplit[n])]
    #display(entry)
    n += 1
    
n = 0    
for entry in job_cat18_list:
    job_cat18_female_df_dict[entry] = df18[list(job_cat18_female_varsplit[n])]
    #display(entry)
    n += 1
    
q1_1_18_tup_male = []
q1_1_18_tup_female = []

# Returns a list of States from the original dataframe.
df18_idx = sorted(list(set(df18.index.get_level_values(0))))

# Create a tuple with "state, key, sum" then convert into dataframe.
for key, value in job_cat18_male_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df18_idx:
        q1_1_18_tup_male.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

for key, value in job_cat18_female_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df18_idx:
        q1_1_18_tup_female.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

# Create dataframe based on tuple.
q1_1_18_male = pd.DataFrame(q1_1_18_tup_male, columns = ['STATE_LABEL', 'Job Category', 'Male Count'])
q1_1_18_male = q1_1_18_male.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_18_female = pd.DataFrame(q1_1_18_tup_female, columns = ['STATE_LABEL', 'Job Category', 'Female Count'])
q1_1_18_female = q1_1_18_female.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_18 = pd.merge(q1_1_18_male, q1_1_18_female, on = ['STATE_LABEL', 'Job Category'])     
q1_1_18 = q1_1_18.set_index(['STATE_LABEL', 'Job Category'])
q1_1_18['Total Count'] = q1_1_18.sum(axis = 1)
q1_1_18['% Men'] = q1_1_18['Male Count']/q1_1_18['Total Count']
q1_1_18['% Women'] = q1_1_18['Female Count']/q1_1_18['Total Count']

#display_side_by_side([q1_1_18_male, q1_1_18_female], ['Male Count in Each Job Category', 'Female Count in Each Job Category'])
display(q1_1_18.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))
#display(job_cat18_male_varsplit)
display(job_cat18_male_varsplit)
#display(job_gen18_male.style.set_caption('Job Categories by Gender'))

### Question 2.0.

In [ ]:
race_ind18_var = []

for k in race_list:
    for i, j in data_dict18.iterrows():
        if j[2] == k:
            race_ind18_var.append(j[0])

race_ind18_varsplit = np.array_split(race_ind18_var, 7)
race_ind18_df_dict = {}

q2_0_18 = pd.DataFrame(index = df18.index)

n = 0
for race in race_list:
    race_ind18_df_dict[race] = df18[list(race_ind18_varsplit[n])]
    n += 1

for key, value in race_ind18_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum by Industry and Race'] = value.sum(axis = 1)
    #display(key, value['Sum by Industry and Race'])
    q2_0_18[key] = value['Sum by Industry and Race']
    
q2_0_18['NAC3 Code Description'] = df18['NAC3_LABEL']  
q2_0_18_col_header = ['NAC3 Code Description', 'AM-INDIAN-ALK', 'ASIAN', 'BLACK', 'HAWAIIAN', 'HISPANIC', 'TWO OR MORE RACES', 'WHITE']
q2_0_18 = q2_0_18[q2_0_18_col_header]
q2_0_18 = q2_0_18.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })

q2_0_18['Total Count'] = q2_0_18.sum(axis = 1)
q2_0_18['% American Indian'] = q2_0_18['AM-INDIAN-ALK Count']/q2_0_18['Total Count']
q2_0_18['% Asian'] = q2_0_18['ASIAN Count']/q2_0_18['Total Count']
q2_0_18['% Black'] = q2_0_18['BLACK Count']/q2_0_18['Total Count']
q2_0_18['% Hawaiian'] = q2_0_18['HAWAIIAN Count']/q2_0_18['Total Count']
q2_0_18['% Hispanic'] = q2_0_18['HISPANIC Count']/q2_0_18['Total Count']
q2_0_18['% Two or More Races'] = q2_0_18['TWO OR MORE RACES Count']/q2_0_18['Total Count']
q2_0_18['% White'] = q2_0_18['WHITE Count']/q2_0_18['Total Count']

display(q2_0_18.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))

### Question 2.1.

In [ ]:
job_race18_list = []
job_race18_df_dict = {}
job_race18_key = []
job_race18_key_unq = []
job_race18_var = []

df18_idx = sorted(list(set(df18.index.get_level_values(0))))

q2_1_18_tup = []
 
n = 0
for race in race_list:
    for job in job_cat18_list:
        for i, j in data_dict18.iterrows():
            if (j[1] == job) and (j[2] == race):
                job_race18_list.append(j[0])
                job_race18_key.append((n, race + ', ' + job))
                n += 1

job_race18_arr = np.array_split(job_race18_list, 70)

for arr in job_race18_arr:
    job_race18_var.append(arr.tolist())

for i in job_race18_key:
    if i[0] % 2 == 0:
        job_race18_key_unq.append(i[1])
        
kv = list(zip(job_race18_key_unq, job_race18_var))

for i, j in kv:
    job_race18_df_dict[i] = df18[j]
    
for key, value in job_race18_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum of Race by Job Category'] = value.sum(axis = 1)
    #display(key, value)
    for state in df18_idx:
        q2_1_18_tup.append(tuple((state, key.split(',')[0],key.split(',')[1].strip(), value.loc[state]['Sum of Race by Job Category'].sum())))
        
# Use the multilevel index from question 1.1 and column headers to be the race list to set up empty dataframe. Iterate through the q2_1_18_tup list to check what race then use
# q2_1_18.loc[state, job category][race] = q2_1_18_tup[3]
q2_1_18 = pd.DataFrame(index = q1_1_18.index, columns = race_list)  

for tup in q2_1_18_tup:
    q2_1_18.loc[tup[0], tup[2]][tup[1]] = tup[3]
    
q2_1_18 = q2_1_18.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })
    
q2_1_18['Total Count'] = q2_1_18.sum(axis = 1)
q2_1_18['% American Indian'] = q2_1_18['AM-INDIAN-ALK Count']/q2_1_18['Total Count']
q2_1_18['% Asian'] = q2_1_18['ASIAN Count']/q2_1_18['Total Count']
q2_1_18['% Black'] = q2_1_18['BLACK Count']/q2_1_18['Total Count']
q2_1_18['% Hawaiian'] = q2_1_18['HAWAIIAN Count']/q2_1_18['Total Count']
q2_1_18['% Hispanic'] = q2_1_18['HISPANIC Count']/q2_1_18['Total Count']
q2_1_18['% Two or More Races'] = q2_1_18['TWO OR MORE RACES Count']/q2_1_18['Total Count']
q2_1_18['% White'] = q2_1_18['WHITE Count']/q2_1_18['Total Count']

display(q2_1_18.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))


# YEAR 17

In [ ]:
with open('year17_state_nac3.txt') as csvfile:
    data17 = csv.reader(csvfile, delimiter = ';')
    df17 = pd.DataFrame(data17)

header17 = df17.iloc[0]
df17 = df17.iloc[1:]
df17.columns = header17
df17 = df17.set_index(['STATE_LABEL', 'NAC3'])
df17 = df17.replace('', np.nan)

data_dict17 = pd.read_excel(io = 'data_dictionary-2017.xls', sheet_name = 'state_NAIC3_2016')
data_dict17 = data_dict17.drop(data_dict17.columns[[0, 3, 4]], axis = 1)
data_dict17 = data_dict17.iloc[3:144]
data_dcit17 = data_dict17.reset_index(drop = True, inplace = True)
data_dict17.columns = ['Variable', 'Label']
data_dict17['Job Category'], data_dict17['Race'], data_dict17['Gender'] = data_dict17['Label'].replace('[0-9.-]', '').str.split('/', 2).str
data_dict17 = data_dict17.drop('Label', axis = 1)

gen_list = ['MALE', 'FEMALE']
race_list = ['WHITE', 'HISPANIC', 'BLACK', 'ASIAN', 'AM-INDIAN-ALK', 'HAWAIIAN', 'TWO OR MORE RACES']

df17_err_gen = data_dict17[~data_dict17['Gender'].isin(gen_list)]
df17_errcor_gen = df17_err_gen.index.to_list()

df17_err_race = data_dict17[~data_dict17['Race'].isin(race_list)]
df17_errcor_race = df17_err_race.index.to_list()

df17_err_race_hwn = df17_err_race[df17_err_race['Race'].str.upper().str.contains('HAWAI')]
df17_errcor_race_hwn = df17_err_race_hwn.index.to_list()

df17_err_race_asn = df17_err_race[df17_err_race['Race'].str.upper().str.contains('ASIA')]
df17_errcor_race_asn = df17_err_race_asn.index.to_list()

df17_err_race_lat = df17_err_race[df17_err_race['Race'].str.upper().str.contains('HISP')]
df17_errcor_race_lat = df17_err_race_lat.index.to_list()

df17_err_race_tmr = df17_err_race[df17_err_race['Race'].str.upper().str.contains('TWO')]
df17_errcor_race_tmr = df17_err_race_tmr.index.to_list()

data_dict17.loc[51, 'Gender'] = 'FEMALE'
data_dict17.loc[61, 'Gender'] = 'MALE'
data_dict17.loc[71, 'Gender'] = 'FEMALE'

data_dict17.loc[df17_errcor_race_hwn, 'Race'] = 'HAWAIIAN'
data_dict17.loc[df17_errcor_race_asn, 'Race'] = 'ASIAN'
data_dict17.loc[df17_errcor_race_lat, 'Race'] = 'HISPANIC'
data_dict17.loc[df17_errcor_race_tmr, 'Race'] = 'TWO OR MORE RACES'

#display(df17_errcor_race_lat)
#display(df17_err_race_lat)
display(data_dict17.style.set_caption('2017 Data Dictionary'))
display(df17)


### Question 1.0. 

In [ ]:
data_dict17_male = data_dict17[(data_dict17['Gender'] == 'MALE')]
data_dict17_female = data_dict17[(data_dict17['Gender'] == 'FEMALE')]
#display(data_dict17_female)
#display_side_by_side([data_dict17_male, data_dict17_female], ['Variables by Men', 'Variables by Women'] )

men17 = data_dict17_male['Variable'].to_list()
women17 = data_dict17_female['Variable'].to_list()

df17_ind_men = df17[men17]
df17_ind_women = df17[women17]

df17_ind_men = df17_ind_men.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df17_ind_men["Men's Count"] = df17_ind_men.sum(axis = 1)

df17_ind_women = df17_ind_women.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df17_ind_women["Women's Count"] = df17_ind_women.sum(axis = 1)

q1_0_17 = pd.DataFrame(index = df17.index)
q1_0_17["Male Count"] = df17_ind_men["Men's Count"]
q1_0_17["Female Count"] = df17_ind_women["Women's Count"]
q1_0_17['Total Count'] = q1_0_17.sum(axis = 1)
q1_0_17['% Men'] = (q1_0_17['Male Count']/q1_0_17['Total Count'])
q1_0_17['% Women'] = (q1_0_17['Female Count']/q1_0_17['Total Count'])
q1_0_17['NAC3 Code Description'] = df17['NAC3_LABEL']
col_head17 = ['NAC3 Code Description', 'Male Count', 'Female Count', 'Total Count', '% Men', '% Women']
q1_0_17 = q1_0_17[col_head17]

#display(len(q1_0_17.groupby(level = 0)))
display(q1_0_17.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))

#display(df17_ind_men, df17_ind_women)


### Question 1.1. 

In [ ]:
# Returns the 10 job categories in the dataset.
job_cat17_list = data_dict17.loc[1:10, 'Job Category'].to_list()

# Corrects the slight mismatch of job categories.
for i, j in data_dict17.iterrows():
    for k in job_cat17_list:
        if j[1][:2] == k[:2]:
            data_dict17.loc[i, 'Job Category'] = k
            
# List to store the variables corresponding to job category for each race.
job_cat17_male_var = []    
job_cat17_female_var = []    

# Extracting the "variable" element from male data dictionary based on the job category matching across alll races
# then adding to a list.
for k in job_cat17_list:
    for i, j in data_dict17.iterrows():
        if (k == j[1]) and (j[3] == 'MALE'):
            job_cat17_male_var.append(j[0])

for k in job_cat17_list:
    for i, j in data_dict17.iterrows():
        if (k == j[1]) and (j[3] == 'FEMALE'):
            job_cat17_female_var.append(j[0])            
            
# Splits the entire list into 10 arrays of size 7 grouping together the job category across all races.
job_cat17_male_varsplit = np.array_split(job_cat17_male_var, 10)
job_cat17_female_varsplit = np.array_split(job_cat17_female_var, 10)

# Initialize a dictionary where I store all the generated dataframes based on the job category for all races.
job_cat17_male_df_dict = {}
job_cat17_female_df_dict = {}

# Fills the dictionary and replaces all empty cells within each dataframe with an NaN value.
n = 0
for entry in job_cat17_list:
    job_cat17_male_df_dict[entry] = df17[list(job_cat17_male_varsplit[n])]
    #display(entry)
    n += 1
    
n = 0    
for entry in job_cat17_list:
    job_cat17_female_df_dict[entry] = df17[list(job_cat17_female_varsplit[n])]
    #display(entry)
    n += 1
    
q1_1_17_tup_male = []
q1_1_17_tup_female = []

# Returns a list of States from the original dataframe.
df17_idx = sorted(list(set(df17.index.get_level_values(0))))

# Create a tuple with "state, key, sum" then convert into dataframe.
for key, value in job_cat17_male_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df17_idx:
        q1_1_17_tup_male.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

for key, value in job_cat17_female_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df17_idx:
        q1_1_17_tup_female.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

# Create dataframe based on tuple.
q1_1_17_male = pd.DataFrame(q1_1_17_tup_male, columns = ['STATE_LABEL', 'Job Category', 'Male Count'])
q1_1_17_male = q1_1_17_male.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_17_female = pd.DataFrame(q1_1_17_tup_female, columns = ['STATE_LABEL', 'Job Category', 'Female Count'])
q1_1_17_female = q1_1_17_female.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_17 = pd.merge(q1_1_17_male, q1_1_17_female, on = ['STATE_LABEL', 'Job Category'])     
q1_1_17 = q1_1_17.set_index(['STATE_LABEL', 'Job Category'])
q1_1_17['Total Count'] = q1_1_17.sum(axis = 1)
q1_1_17['% Men'] = q1_1_17['Male Count']/q1_1_17['Total Count']
q1_1_17['% Women'] = q1_1_17['Female Count']/q1_1_17['Total Count']

#display_side_by_side([q1_1_17_male, q1_1_17_female], ['Male Count in Each Job Category', 'Female Count in Each Job Category'])
display(q1_1_17.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))
#display(job_cat17_male_varsplit)
display(job_cat17_male_varsplit)
#display(job_gen17_male.style.set_caption('Job Categories by Gender'))

### Question 2.0.

In [ ]:
race_ind17_var = []

for k in race_list:
    for i, j in data_dict17.iterrows():
        if j[2] == k:
            race_ind17_var.append(j[0])

race_ind17_varsplit = np.array_split(race_ind17_var, 7)
race_ind17_df_dict = {}

q2_0_17 = pd.DataFrame(index = df17.index)

n = 0
for race in race_list:
    race_ind17_df_dict[race] = df17[list(race_ind17_varsplit[n])]
    n += 1

for key, value in race_ind17_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum by Race'] = value.sum(axis = 1)
    #display(key, value['Sum by Race'])
    q2_0_17[key] = value['Sum by Race']
    
q2_0_17['NAC3 Code Description'] = df17['NAC3_LABEL']  
q2_0_17_col_header = ['NAC3 Code Description', 'AM-INDIAN-ALK', 'ASIAN', 'BLACK', 'HAWAIIAN', 'HISPANIC', 'TWO OR MORE RACES', 'WHITE']
q2_0_17 = q2_0_17[q2_0_17_col_header]
q2_0_17 = q2_0_17.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })

q2_0_17['Total Count'] = q2_0_17.sum(axis = 1)
q2_0_17['% American Indian'] = q2_0_17['AM-INDIAN-ALK Count']/q2_0_17['Total Count']
q2_0_17['% Asian'] = q2_0_17['ASIAN Count']/q2_0_17['Total Count']
q2_0_17['% Black'] = q2_0_17['BLACK Count']/q2_0_17['Total Count']
q2_0_17['% Hawaiian'] = q2_0_17['HAWAIIAN Count']/q2_0_17['Total Count']
q2_0_17['% Hispanic'] = q2_0_17['HISPANIC Count']/q2_0_17['Total Count']
q2_0_17['% Two or More Races'] = q2_0_17['TWO OR MORE RACES Count']/q2_0_17['Total Count']
q2_0_17['% White'] = q2_0_17['WHITE Count']/q2_0_17['Total Count']

display(q2_0_17.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))

### Question 2.1.

In [ ]:
job_race17_list = []
job_race17_df_dict = {}
job_race17_key = []
job_race17_key_unq = []
job_race17_var = []

df17_idx = sorted(list(set(df17.index.get_level_values(0))))

q2_1_17_tup = []
 
n = 0
for race in race_list:
    for job in job_cat17_list:
        for i, j in data_dict17.iterrows():
            if (j[1] == job) and (j[2] == race):
                job_race17_list.append(j[0])
                job_race17_key.append((n, race + ', ' + job))
                n += 1

job_race17_arr = np.array_split(job_race17_list, 70)

for arr in job_race17_arr:
    job_race17_var.append(arr.tolist())

for i in job_race17_key:
    if i[0] % 2 == 0:
        job_race17_key_unq.append(i[1])
        
kv = list(zip(job_race17_key_unq, job_race17_var))

for i, j in kv:
    job_race17_df_dict[i] = df17[j]
    
for key, value in job_race17_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum of Race by Job Category'] = value.sum(axis = 1)
    #display(key, value)
    for state in df17_idx:
        q2_1_17_tup.append(tuple((state, key.split(',')[0],key.split(',')[1].strip(), value.loc[state]['Sum of Race by Job Category'].sum())))
        
# Use the multilevel index from question 1.1 and column headers to be the race list to set up empty dataframe. Iterate through the q2_1_17_tup list to check what race then use
# q2_1_17.loc[state, job category][race] = q2_1_17_tup[3]
q2_1_17 = pd.DataFrame(index = q1_1_17.index, columns = race_list)  

for tup in q2_1_17_tup:
    q2_1_17.loc[tup[0], tup[2]][tup[1]] = tup[3]
    
q2_1_17 = q2_1_17.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })
    
q2_1_17['Total Count'] = q2_1_17.sum(axis = 1)
q2_1_17['% American Indian'] = q2_1_17['AM-INDIAN-ALK Count']/q2_1_17['Total Count']
q2_1_17['% Asian'] = q2_1_17['ASIAN Count']/q2_1_17['Total Count']
q2_1_17['% Black'] = q2_1_17['BLACK Count']/q2_1_17['Total Count']
q2_1_17['% Hawaiian'] = q2_1_17['HAWAIIAN Count']/q2_1_17['Total Count']
q2_1_17['% Hispanic'] = q2_1_17['HISPANIC Count']/q2_1_17['Total Count']
q2_1_17['% Two or More Races'] = q2_1_17['TWO OR MORE RACES Count']/q2_1_17['Total Count']
q2_1_17['% White'] = q2_1_17['WHITE Count']/q2_1_17['Total Count']

display(q2_1_17.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))


## YEAR 16

In [ ]:
with open('year16_state_nac3.txt') as csvfile:
    data16 = csv.reader(csvfile, delimiter = ';')
    df16 = pd.DataFrame(data16)

header16 = df16.iloc[0]
df16 = df16.iloc[1:]
df16.columns = header16
df16 = df16.set_index(['STATE_LABEL', 'NAC3'])
df16 = df16.replace('', np.nan)

data_dict16 = pd.read_excel(io = 'data_dictionary-2016.xls', sheet_name = 'state_NAIC3_2016')
data_dict16 = data_dict16.drop(data_dict16.columns[[0, 3, 4]], axis = 1)
data_dict16 = data_dict16.iloc[3:143]
data_dcit16 = data_dict16.reset_index(drop = True, inplace = True)
data_dict16.columns = ['Variable', 'Label']
data_dict16['Job Category'], data_dict16['Race'], data_dict16['Gender'] = data_dict16['Label'].replace('[0-9.-]', '').str.split('/', 2).str
data_dict16 = data_dict16.drop('Label', axis = 1)

with open('year16_state_nac3.txt') as csvfile:
    data16 = csv.reader(csvfile, delimiter = ';')
    df16 = pd.DataFrame(data16)

header16 = df16.iloc[0]
df16 = df16.iloc[1:]
df16.columns = header16
df16 = df16.set_index(['STATE_LABEL', 'NAC3'])
df16 = df16.replace('', np.nan)

data_dict16 = pd.read_excel(io = 'data_dictionary-2016.xls', sheet_name = 'state_NAIC3_2016')
data_dict16 = data_dict16.drop(data_dict16.columns[[0, 3, 4]], axis = 1)
data_dict16 = data_dict16.iloc[3:143]
data_dcit16 = data_dict16.reset_index(drop = True, inplace = True)
data_dict16.columns = ['Variable', 'Label']
data_dict16['Job Category'], data_dict16['Race'], data_dict16['Gender'] = data_dict16['Label'].replace('[0-9.-]', '').str.split('/', 2).str
data_dict16 = data_dict16.drop('Label', axis = 1)

gen_list = ['MALE', 'FEMALE']
race_list = ['WHITE', 'HISPANIC', 'BLACK', 'ASIAN', 'AM-INDIAN-ALK', 'HAWAIIAN', 'TWO OR MORE RACES']

df16_err_gen = data_dict16[~data_dict16['Gender'].isin(gen_list)]
df16_errcor_gen = df16_err_gen.index.to_list()

df16_err_race = data_dict16[~data_dict16['Race'].isin(race_list)]
df16_errcor_race = df16_err_race.index.to_list()

df16_err_race_hwn = df16_err_race[df16_err_race['Race'].str.upper().str.contains('HAWAI')]
df16_errcor_race_hwn = df16_err_race_hwn.index.to_list()

df16_err_race_asn = df16_err_race[df16_err_race['Race'].str.upper().str.contains('ASIA')]
df16_errcor_race_asn = df16_err_race_asn.index.to_list()

df16_err_race_lat = df16_err_race[df16_err_race['Race'].str.upper().str.contains('HISP')]
df16_errcor_race_lat = df16_err_race_lat.index.to_list()

df16_err_race_tmr = df16_err_race[df16_err_race['Race'].str.upper().str.contains('TWO')]
df16_errcor_race_tmr = df16_err_race_tmr.index.to_list()

data_dict16.loc[51, 'Gender'] = 'FEMALE'
data_dict16.loc[61, 'Gender'] = 'MALE'
data_dict16.loc[71, 'Gender'] = 'FEMALE'

data_dict16.loc[df16_errcor_race_hwn, 'Race'] = 'HAWAIIAN'
data_dict16.loc[df16_errcor_race_asn, 'Race'] = 'ASIAN'
data_dict16.loc[df16_errcor_race_lat, 'Race'] = 'HISPANIC'
data_dict16.loc[df16_errcor_race_tmr, 'Race'] = 'TWO OR MORE RACES'

#display(df16_errcor_race_tmr)
#display(df16_err_race_tmr)
display(data_dict16.style.set_caption('2016 Data Dictionary'))
display(df16)


### Question 1.0. 

In [ ]:
data_dict16_male = data_dict16[(data_dict16['Gender'] == 'MALE')]
data_dict16_female = data_dict16[(data_dict16['Gender'] == 'FEMALE')]

#display_side_by_side([data_dict17_male, data_dict17_female], ['Variables by Men', 'Variables by Women'] )

men16 = data_dict16_male['Variable'].to_list()
women16 = data_dict16_female['Variable'].to_list()

df16_ind_men = df16[men16]
df16_ind_women = df16[women16]

df16_ind_men = df16_ind_men.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df16_ind_men["Men's Count"] = df16_ind_men.sum(axis = 1)

df16_ind_women = df16_ind_women.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df16_ind_women["Women's Count"] = df16_ind_women.sum(axis = 1)

q1_0_16 = pd.DataFrame(index = df16.index)
q1_0_16["Male Count"] = df16_ind_men["Men's Count"]
q1_0_16["Female Count"] = df16_ind_women["Women's Count"]
q1_0_16['Total Count'] = q1_0_16.sum(axis = 1)
q1_0_16['% Men'] = (q1_0_16['Male Count']/q1_0_16['Total Count'])
q1_0_16['% Women'] = (q1_0_16['Female Count']/q1_0_16['Total Count'])
q1_0_16['NAC3 Code Description'] = df16['NAC3_LABEL']
col_head16 = ['NAC3 Code Description', 'Male Count', 'Female Count', 'Total Count', '% Men', '% Women']
q1_0_16 = q1_0_16[col_head16]

#display(len(q1_0_16.groupby(level = 0)))
display(q1_0_16.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))

#display(df16_ind_men, df16_ind_women)

### Question 1.1. 

In [ ]:
# Returns the 10 job categories in the dataset.
job_cat16_list = data_dict16.loc[1:10, 'Job Category'].to_list()

# Corrects the slight mismatch of job categories.
for i, j in data_dict16.iterrows():
    for k in job_cat16_list:
        if j[1][:2] == k[:2]:
            data_dict16.loc[i, 'Job Category'] = k
            
# List to store the variables corresponding to job category for each race.
job_cat16_male_var = []    
job_cat16_female_var = []    

# Extracting the "variable" element from male data dictionary based on the job category matching across alll races
# then adding to a list.
for k in job_cat16_list:
    for i, j in data_dict16.iterrows():
        if (k == j[1]) and (j[3] == 'MALE'):
            job_cat16_male_var.append(j[0])

for k in job_cat16_list:
    for i, j in data_dict16.iterrows():
        if (k == j[1]) and (j[3] == 'FEMALE'):
            job_cat16_female_var.append(j[0])            
            
# Splits the entire list into 10 arrays of size 7 grouping together the job category across all races.
job_cat16_male_varsplit = np.array_split(job_cat16_male_var, 10)
job_cat16_female_varsplit = np.array_split(job_cat16_female_var, 10)

# Initialize a dictionary where I store all the generated dataframes based on the job category for all races.
job_cat16_male_df_dict = {}
job_cat16_female_df_dict = {}

# Fills the dictionary and replaces all empty cells within each dataframe with an NaN value.
n = 0
for entry in job_cat16_list:
    job_cat16_male_df_dict[entry] = df16[list(job_cat16_male_varsplit[n])]
    #display(entry)
    n += 1
    
n = 0    
for entry in job_cat16_list:
    job_cat16_female_df_dict[entry] = df16[list(job_cat16_female_varsplit[n])]
    #display(entry)
    n += 1
    
q1_1_16_tup_male = []
q1_1_16_tup_female = []

# Returns a list of States from the original dataframe.
df16_idx = sorted(list(set(df16.index.get_level_values(0))))

# Create a tuple with "state, key, sum" then convert into dataframe.
for key, value in job_cat16_male_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df16_idx:
        q1_1_16_tup_male.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

for key, value in job_cat16_female_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df16_idx:
        q1_1_16_tup_female.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

# Create dataframe based on tuple.
q1_1_16_male = pd.DataFrame(q1_1_16_tup_male, columns = ['STATE_LABEL', 'Job Category', 'Male Count'])
q1_1_16_male = q1_1_16_male.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_16_female = pd.DataFrame(q1_1_16_tup_female, columns = ['STATE_LABEL', 'Job Category', 'Female Count'])
q1_1_16_female = q1_1_16_female.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_16 = pd.merge(q1_1_16_male, q1_1_16_female, on = ['STATE_LABEL', 'Job Category'])     
q1_1_16 = q1_1_16.set_index(['STATE_LABEL', 'Job Category'])
q1_1_16['Total Count'] = q1_1_16.sum(axis = 1)
q1_1_16['% Men'] = q1_1_16['Male Count']/q1_1_16['Total Count']
q1_1_16['% Women'] = q1_1_16['Female Count']/q1_1_16['Total Count']

#display_side_by_side([q1_1_16_male, q1_1_16_female], ['Male Count in Each Job Category', 'Female Count in Each Job Category'])
display(q1_1_16.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))
#display(job_cat16_male_varsplit)
display(job_cat16_male_varsplit)
#display(job_gen16_male.style.set_caption('Job Categories by Gender'))

### Question 2.0.

In [ ]:
race_ind16_var = []

for k in race_list:
    for i, j in data_dict16.iterrows():
        if j[2] == k:
            race_ind16_var.append(j[0])

race_ind16_varsplit = np.array_split(race_ind16_var, 7)
race_ind16_df_dict = {}

q2_0_16 = pd.DataFrame(index = df16.index)

n = 0
for race in race_list:
    race_ind16_df_dict[race] = df16[list(race_ind16_varsplit[n])]
    n += 1

for key, value in race_ind16_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum by Race'] = value.sum(axis = 1)
    #display(key, value['Sum by Race'])
    q2_0_16[key] = value['Sum by Race']
    
q2_0_16['NAC3 Code Description'] = df16['NAC3_LABEL']  
q2_0_16_col_header = ['NAC3 Code Description', 'AM-INDIAN-ALK', 'ASIAN', 'BLACK', 'HAWAIIAN', 'HISPANIC', 'TWO OR MORE RACES', 'WHITE']
q2_0_16 = q2_0_16[q2_0_16_col_header]
q2_0_16 = q2_0_16.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })

q2_0_16['Total Count'] = q2_0_16.sum(axis = 1)
q2_0_16['% American Indian'] = q2_0_16['AM-INDIAN-ALK Count']/q2_0_16['Total Count']
q2_0_16['% Asian'] = q2_0_16['ASIAN Count']/q2_0_16['Total Count']
q2_0_16['% Black'] = q2_0_16['BLACK Count']/q2_0_16['Total Count']
q2_0_16['% Hawaiian'] = q2_0_16['HAWAIIAN Count']/q2_0_16['Total Count']
q2_0_16['% Hispanic'] = q2_0_16['HISPANIC Count']/q2_0_16['Total Count']
q2_0_16['% Two or More Races'] = q2_0_16['TWO OR MORE RACES Count']/q2_0_16['Total Count']
q2_0_16['% White'] = q2_0_16['WHITE Count']/q2_0_16['Total Count']

display(q2_0_16.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))

### Question 2.1.

In [ ]:
job_race16_list = []
job_race16_df_dict = {}
job_race16_key = []
job_race16_key_unq = []
job_race16_var = []

df16_idx = sorted(list(set(df16.index.get_level_values(0))))

q2_1_16_tup = []
 
n = 0
for race in race_list:
    for job in job_cat16_list:
        for i, j in data_dict16.iterrows():
            if (j[1] == job) and (j[2] == race):
                job_race16_list.append(j[0])
                job_race16_key.append((n, race + ', ' + job))
                n += 1

job_race16_arr = np.array_split(job_race16_list, 70)

for arr in job_race16_arr:
    job_race16_var.append(arr.tolist())

for i in job_race16_key:
    if i[0] % 2 == 0:
        job_race16_key_unq.append(i[1])
        
kv = list(zip(job_race16_key_unq, job_race16_var))

for i, j in kv:
    job_race16_df_dict[i] = df16[j]
    
for key, value in job_race16_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum of Race by Job Category'] = value.sum(axis = 1)
    #display(key, value)
    for state in df16_idx:
        q2_1_16_tup.append(tuple((state, key.split(',')[0],key.split(',')[1].strip(), value.loc[state]['Sum of Race by Job Category'].sum())))
        
# Use the multilevel index from question 1.1 and column headers to be the race list to set up empty dataframe. Iterate through the q2_1_16_tup list to check what race then use
# q2_1_16.loc[state, job category][race] = q2_1_16_tup[3]
q2_1_16 = pd.DataFrame(index = q1_1_16.index, columns = race_list)  

for tup in q2_1_16_tup:
    q2_1_16.loc[tup[0], tup[2]][tup[1]] = tup[3]
    
q2_1_16 = q2_1_16.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })
    
q2_1_16['Total Count'] = q2_1_16.sum(axis = 1)
q2_1_16['% American Indian'] = q2_1_16['AM-INDIAN-ALK Count']/q2_1_16['Total Count']
q2_1_16['% Asian'] = q2_1_16['ASIAN Count']/q2_1_16['Total Count']
q2_1_16['% Black'] = q2_1_16['BLACK Count']/q2_1_16['Total Count']
q2_1_16['% Hawaiian'] = q2_1_16['HAWAIIAN Count']/q2_1_16['Total Count']
q2_1_16['% Hispanic'] = q2_1_16['HISPANIC Count']/q2_1_16['Total Count']
q2_1_16['% Two or More Races'] = q2_1_16['TWO OR MORE RACES Count']/q2_1_16['Total Count']
q2_1_16['% White'] = q2_1_16['WHITE Count']/q2_1_16['Total Count']

display(q2_1_16.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))


## YEAR 15 

In [ ]:
with open('year15_state_nac3.txt') as csvfile:
    data15 = csv.reader(csvfile, delimiter = ';')
    df15 = pd.DataFrame(data15)

header15 = df15.iloc[0]
df15 = df15.iloc[1:]
df15.columns = header15
df15 = df15.set_index(['STATE_LABEL', 'NAC3'])
df15 = df15.replace([None], np.nan)

data_dict15 = pd.read_excel(io = 'data_dictionary-2013.xls', sheet_name = 'state_NAICS3')
data_dict15 = data_dict15.drop(data_dict15.columns[[0, 3, 4]], axis = 1)
data_dict15 = data_dict15.iloc[2:598]
data_dcit15 = data_dict15.reset_index(drop = True, inplace = True)
data_dict15.columns = ['Variable', 'Label']
data_dict15['Job Category'], data_dict15['Race'], data_dict15['Gender'] = data_dict15['Label'].replace('[0-9.-]', '').str.split('/', 2).str
data_dict15 = data_dict15.drop('Label', axis = 1)

# Using the .replace() method is straightforward since the variations for the two races are consistent.
data_dict15['Race'] = data_dict15['Race'].replace('Hawaiia', 'HAWAIIAN')
data_dict15['Race'] = data_dict15['Race'].replace(dict.fromkeys(['Two Races', 'Two More Races'], 'TWO OR MORE RACES'))

# What if the errors from the data dictionary for race categories aren't consistent? I use .str.contains() method to search
# "Race" column for a partial match on a race.
lat_df15 = data_dict15[data_dict15['Race'].str.contains('HISPANIC', na = False)]
asn_df15 = data_dict15[data_dict15['Race'].str.contains('ASIAN|Asian', na = False)]

# I return a dataframe from a slice where the conditional statement only gives entries where there are mismatches from the preferred variable.
asn_errcor15 = asn_df15[(asn_df15['Race'] != 'ASIAN')].copy()
lat_errcor15 = lat_df15[(lat_df15['Race'] != 'HISPANIC')].copy()

# Correcting the dataframe for each race category. This method is useful if there are many entries with the same mistake.
asn_errcor15['Race'] = 'ASIAN'
lat_errcor15['Race'] = 'HISPANIC'

lat_errcor15.loc[122, 'Gender'] = 'FEMALE'

# Updating the data dictionary to be consistent with the preferred variable names for race using original dataframe indices to match.
# Instead of updating each one by row and column, I update the original dataframe with a dataframe comprised of corrected entries.
data_dict15.update(asn_errcor15, overwrite = True)
data_dict15.update(lat_errcor15, overwrite = True)

gen_errcor15 = data_dict15[(data_dict15['Gender'] != 'MALE') & (data_dict15['Gender'] != 'FEMALE') & (data_dict15['Gender'].notnull()) ]
gen_errcor15 = gen_errcor15[~(gen_errcor15['Gender'].str.contains('MINORITY') | gen_errcor15['Gender'].str.contains('PERCENT'))]

data_dict15.loc[144, 'Gender'] = 'MALE'
data_dict15.loc[155, 'Gender'] = 'FEMALE'
data_dict15.loc[354, 'Gender'] = 'MALE(PERCENT)'

#display(gen_errcor15.style.set_caption('Gender Correction'))
#display_side_by_side([asn_errcor15, lat_errcor15], ['Dataframe Error Correction Asian','Dataframe Error Correction Hispanic'])
#display_side_by_side([asn_df15, lat_df15], ['Dataframe Only Asian Race', 'Dataframe Only Hispanic Race'])
display(data_dict15.style.set_caption('2015 Data Dictionary'))
display(df15)

### Question 1.0. 

In [ ]:
data_dict15_male = data_dict15[(data_dict15['Gender'] == 'MALE') & (data_dict15['Job Category'] != '10-TOTAL' )]
data_dict15_female = data_dict15[(data_dict15['Gender'] == 'FEMALE') & (data_dict15['Job Category'] != '10-TOTAL' )]

men15 = data_dict15_male['Variable'].to_list()
women15 = data_dict15_female['Variable'].to_list()

df15_ind_men = df15[men15]
df15_ind_women = df15[women15]

df15_ind_men = df15_ind_men.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df15_ind_men["Men's Count"] = df15_ind_men.sum(axis = 1)

df15_ind_women = df15_ind_women.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df15_ind_women["Women's Count"] = df15_ind_women.sum(axis = 1)

q1_0_15 = pd.DataFrame(index = df15.index)
q1_0_15["Male Count"] = df15_ind_men["Men's Count"]
q1_0_15["Female Count"] = df15_ind_women["Women's Count"]
q1_0_15['Total Count'] = q1_0_15.sum(axis = 1)
q1_0_15['% Men'] = (q1_0_15['Male Count']/q1_0_15['Total Count'])
q1_0_15['% Women'] = (q1_0_15['Female Count']/q1_0_15['Total Count'])
q1_0_15['NAC3 Code Description'] = df15['NAC3_LABEL']
col_head15 = ['NAC3 Code Description', 'Male Count', 'Female Count', 'Total Count', '% Men', '% Women']
q1_0_15 = q1_0_15[col_head15]

#display(len(q1_0_15.groupby(level = 0)))
display(q1_0_15.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))

#display(df15_ind_men, df15_ind_women)


### Question 1.1.

In [ ]:
# Included 10-TOTAL. sort it out. Double check all other years before Y15.

# Returns the 10 job categories in the dataset.
job_cat15_list = data_dict15.loc[1:11, 'Job Category']
job_cat15_list = job_cat15_list[job_cat15_list != '10-TOTAL'].to_list()

# Corrects the slight mismatch of job categories.
for i, j in data_dict15.iterrows():
    for k in job_cat15_list:
        if j[1][:2] == k[:2]:
            data_dict15.loc[i, 'Job Category'] = k
            
# List to store the variables corresponding to job category for each race.
job_cat15_male_var = []    
job_cat15_female_var = []    

# Extracting the "variable" element from male data dictionary based on the job category matching across alll races
# then adding to a list.
for k in job_cat15_list:
    for i, j in data_dict15.iterrows():
        if (k == j[1]) and (j[3] == 'MALE'):
            job_cat15_male_var.append(j[0])

for k in job_cat15_list:
    for i, j in data_dict15.iterrows():
        if (k == j[1]) and (j[3] == 'FEMALE'):
            job_cat15_female_var.append(j[0])            
         
# Splits the entire list into 10 arrays of size 7 grouping together the job category across all races.
job_cat15_male_varsplit = np.array_split(job_cat15_male_var, 10)
job_cat15_female_varsplit = np.array_split(job_cat15_female_var, 10)
 
# Initialize a dictionary where I store all the generated dataframes based on the job category for all races.
job_cat15_male_df_dict = {}
job_cat15_female_df_dict = {}

# Fills the dictionary and replaces all empty cells within each dataframe with an NaN value.
n = 0
for entry in job_cat15_list:
    job_cat15_male_df_dict[entry] = df15[list(job_cat15_male_varsplit[n])]
    #display(entry)
    n += 1
    
n = 0    
for entry in job_cat15_list:
    job_cat15_female_df_dict[entry] = df15[list(job_cat15_female_varsplit[n])]
    #display(entry)
    n += 1
    
q1_1_15_tup_male = []
q1_1_15_tup_female = []

# Returns a list of States from the original dataframe.
df15_idx = sorted(list(set(df15.index.get_level_values(0))))

# Create a tuple with "state, key, sum" then convert into dataframe.
for key, value in job_cat15_male_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df15_idx:
        q1_1_15_tup_male.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

for key, value in job_cat15_female_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df15_idx:
        q1_1_15_tup_female.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

# Create dataframe based on tuple.
q1_1_15_male = pd.DataFrame(q1_1_15_tup_male, columns = ['STATE_LABEL', 'Job Category', 'Male Count'])
q1_1_15_male = q1_1_15_male.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_15_female = pd.DataFrame(q1_1_15_tup_female, columns = ['STATE_LABEL', 'Job Category', 'Female Count'])
q1_1_15_female = q1_1_15_female.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_15 = pd.merge(q1_1_15_male, q1_1_15_female, on = ['STATE_LABEL', 'Job Category'])     
q1_1_15 = q1_1_15.set_index(['STATE_LABEL', 'Job Category'])
q1_1_15['Total Count'] = q1_1_15.sum(axis = 1)
q1_1_15['% Men'] = q1_1_15['Male Count']/q1_1_15['Total Count']
q1_1_15['% Women'] = q1_1_15['Female Count']/q1_1_15['Total Count']

#display_side_by_side([q1_1_15_male, q1_1_15_female], ['Male Count in Each Job Category', 'Female Count in Each Job Category'])
display(q1_1_15.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))
#display(job_cat15_male_varsplit)
#display(job_cat15_male_varsplit)
#display(job_gen15_male.style.set_caption('Job Categories by Gender'))

### Question 2.0.

In [ ]:
race_ind15_var = []

for k in race_list:
    for i, j in data_dict15.iterrows():
        if (j[1] != '10-TOTAL') and (j[2] == k) and (j[3] == 'FEMALE' or j[3] == 'MALE'):
            race_ind15_var.append(j[0])
            
race_ind15_varsplit = np.array_split(race_ind15_var, 7)

race_ind15_df_dict = {}

q2_0_15 = pd.DataFrame(index = df15.index)

n = 0
for race in race_list:
    race_ind15_df_dict[race] = df15[list(race_ind15_varsplit[n])]
    n += 1

for key, value in race_ind15_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum by Race'] = value.sum(axis = 1)
    #display(key, value)
    q2_0_15[key] = value['Sum by Race']
    
q2_0_15['NAC3 Code Description'] = df15['NAC3_LABEL']  
q2_0_15_col_header = ['NAC3 Code Description', 'AM-INDIAN-ALK', 'ASIAN', 'BLACK', 'HAWAIIAN', 'HISPANIC', 'TWO OR MORE RACES', 'WHITE']
q2_0_15 = q2_0_15[q2_0_15_col_header]
q2_0_15 = q2_0_15.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })

q2_0_15['Total Count'] = q2_0_15.sum(axis = 1)
q2_0_15['% American Indian'] = q2_0_15['AM-INDIAN-ALK Count']/q2_0_15['Total Count']
q2_0_15['% Asian'] = q2_0_15['ASIAN Count']/q2_0_15['Total Count']
q2_0_15['% Black'] = q2_0_15['BLACK Count']/q2_0_15['Total Count']
q2_0_15['% Hawaiian'] = q2_0_15['HAWAIIAN Count']/q2_0_15['Total Count']
q2_0_15['% Hispanic'] = q2_0_15['HISPANIC Count']/q2_0_15['Total Count']
q2_0_15['% Two or More Races'] = q2_0_15['TWO OR MORE RACES Count']/q2_0_15['Total Count']
q2_0_15['% White'] = q2_0_15['WHITE Count']/q2_0_15['Total Count']

display(q2_0_15.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))
     


### Question 2.1.

In [ ]:
job_race15_list = []
job_race15_df_dict = {}
job_race15_key = []
job_race15_key_unq = []
job_race15_var = []

df15_idx = sorted(list(set(df15.index.get_level_values(0))))

q2_1_15_tup = []
 
n = 0
for race in race_list:
    for job in job_cat15_list:
        for i, j in data_dict15.iterrows():
            if (j[1] == job) and (j[2] == race) and (j[3] == 'MALE' or j[3] == 'FEMALE'):
                job_race15_list.append(j[0])
                job_race15_key.append((n, race + ', ' + job))
                n += 1

job_race15_arr = np.array_split(job_race15_list, 70)

for arr in job_race15_arr:
    job_race15_var.append(arr.tolist())

for i in job_race15_key:
    if i[0] % 2 == 0:
        job_race15_key_unq.append(i[1])
        
kv = list(zip(job_race15_key_unq, job_race15_var))

for i, j in kv:
    job_race15_df_dict[i] = df15[j]
    
for key, value in job_race15_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum of Race by Job Category'] = value.sum(axis = 1)
    #display(key, value)
    for state in df15_idx:
        q2_1_15_tup.append(tuple((state, key.split(',')[0],key.split(',')[1].strip(), value.loc[state]['Sum of Race by Job Category'].sum())))
        
# Use the multilevel index from question 1.1 and column headers to be the race list to set up empty dataframe. Iterate through the q2_1_15_tup list to check what race then use
# q2_1_15.loc[state, job category][race] = q2_1_15_tup[3]
q2_1_15 = pd.DataFrame(index = q1_1_15.index, columns = race_list)  

for tup in q2_1_15_tup:
    q2_1_15.loc[tup[0], tup[2]][tup[1]] = tup[3]
    
q2_1_15 = q2_1_15.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })
    
q2_1_15['Total Count'] = q2_1_15.sum(axis = 1)
q2_1_15['% American Indian'] = q2_1_15['AM-INDIAN-ALK Count']/q2_1_15['Total Count']
q2_1_15['% Asian'] = q2_1_15['ASIAN Count']/q2_1_15['Total Count']
q2_1_15['% Black'] = q2_1_15['BLACK Count']/q2_1_15['Total Count']
q2_1_15['% Hawaiian'] = q2_1_15['HAWAIIAN Count']/q2_1_15['Total Count']
q2_1_15['% Hispanic'] = q2_1_15['HISPANIC Count']/q2_1_15['Total Count']
q2_1_15['% Two or More Races'] = q2_1_15['TWO OR MORE RACES Count']/q2_1_15['Total Count']
q2_1_15['% White'] = q2_1_15['WHITE Count']/q2_1_15['Total Count']


display(q2_1_15.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))


## YEAR 14 

In [ ]:
with open('year14_state_nac3.txt') as csvfile:
    data14 = csv.reader(csvfile, delimiter = ';')
    df14 = pd.DataFrame(data14)

header14 = df14.iloc[0]
df14 = df14.iloc[1:]
df14.columns = header14
df14 = df14.set_index(['STATE_LABEL', 'NAC3'])
df14 = df14.replace([None], np.nan)

data_dict14 = pd.read_excel(io = 'data_dictionary-2013.xls', sheet_name = 'state_NAICS3')
data_dict14 = data_dict14.drop(data_dict14.columns[[0, 3, 4]], axis = 1)
data_dict14 = data_dict14.iloc[2:598]
data_dcit14 = data_dict14.reset_index(drop = True, inplace = True)
data_dict14.columns = ['Variable', 'Label']
data_dict14['Job Category'], data_dict14['Race'], data_dict14['Gender'] = data_dict14['Label'].replace('[0-9.-]', '').str.split('/', 2).str
data_dict14 = data_dict14.drop('Label', axis = 1)

data_dict14['Race'] = data_dict14['Race'].replace('Hawaiia', 'HAWAIIAN')
data_dict14['Race'] = data_dict14['Race'].replace(dict.fromkeys(['Two Races', 'Two More Races'], 'TWO OR MORE RACES'))

lat_df14 = data_dict14[data_dict14['Race'].str.contains('HISPANIC', na = False)]
asn_df14 = data_dict14[data_dict14['Race'].str.contains('ASIAN|Asian', na = False)]

asn_errcor14 = asn_df14[(asn_df14['Race'] != 'ASIAN')].copy()
lat_errcor14 = lat_df14[(lat_df14['Race'] != 'HISPANIC')].copy()

# Correcting the dataframe for each race category. This method is useful if there are many entries with the same mistake.
asn_errcor14['Race'] = 'ASIAN'
lat_errcor14['Race'] = 'HISPANIC'
lat_errcor14.loc[122, 'Gender'] = 'FEMALE'

# Updating the data dictionary to be consistent with the preferred variable names for race using original dataframe indices to match.
# Instead of updating each one by row and column, I update the original dataframe with a dataframe comprised of corrected entries.
data_dict14.update(asn_errcor14, overwrite = True)
data_dict14.update(lat_errcor14, overwrite = True)

gen_errcor14 = data_dict14[(data_dict14['Gender'] != 'MALE') & (data_dict14['Gender'] != 'FEMALE') & (data_dict14['Gender'].notnull()) ]
gen_errcor14 = gen_errcor14[~(gen_errcor14['Gender'].str.contains('MINORITY') | gen_errcor14['Gender'].str.contains('PERCENT'))]

data_dict14.loc[144, 'Gender'] = 'MALE'
data_dict14.loc[155, 'Gender'] = 'FEMALE'
data_dict14.loc[354, 'Gender'] = 'MALE(PERCENT)'

#display(gen_errcor14.style.set_caption('Gender Correction'))
#display_side_by_side([asn_errcor14, lat_errcor14], ['Dataframe Error Correction Asian','Dataframe Error Correction Hispanic'])
#display_side_by_side([asn_df14, lat_df14], ['Dataframe Only Asian Race', 'Dataframe Only Hispanic Race'])
display(data_dict14.style.set_caption('2014 Data Dictionary'))
display(df14)

### Question 1.0. 

In [ ]:
data_dict14_male = data_dict14[(data_dict14['Gender'] == 'MALE') & (data_dict14['Job Category'] != '10-TOTAL' )]
data_dict14_female = data_dict14[(data_dict14['Gender'] == 'FEMALE') & (data_dict14['Job Category'] != '10-TOTAL' )]

men14 = data_dict14_male['Variable'].to_list()
women14 = data_dict14_female['Variable'].to_list()

df14_ind_men = df14[men14]
df14_ind_women = df14[women14]

df14_ind_men = df14_ind_men.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df14_ind_men["Men's Count"] = df14_ind_men.sum(axis = 1)

df14_ind_women = df14_ind_women.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df14_ind_women["Women's Count"] = df14_ind_women.sum(axis = 1)

q1_0_14 = pd.DataFrame(index = df14.index)
q1_0_14["Male Count"] = df14_ind_men["Men's Count"]
q1_0_14["Female Count"] = df14_ind_women["Women's Count"]
q1_0_14['Total Count'] = q1_0_14.sum(axis = 1)
q1_0_14['% Men'] = (q1_0_14['Male Count']/q1_0_14['Total Count'])
q1_0_14['% Women'] = (q1_0_14['Female Count']/q1_0_14['Total Count'])
q1_0_14['NAC3 Code Description'] = df14['NAC3_LABEL']
col_head14 = ['NAC3 Code Description', 'Male Count', 'Female Count', 'Total Count', '% Men', '% Women']
q1_0_14 = q1_0_14[col_head14]

#display(len(q1_0_14.groupby(level = 0)))
display(q1_0_14.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))

#display(df14_ind_men, df14_ind_women)

### Question 1.1.

In [ ]:
# Returns the 10 job categories in the dataset.
job_cat14_list = data_dict14.loc[1:11, 'Job Category']
job_cat14_list = job_cat14_list[job_cat14_list != '10-TOTAL'].to_list()

# Corrects the slight mismatch of job categories.
for i, j in data_dict14.iterrows():
    for k in job_cat14_list:
        if j[1][:2] == k[:2]:
            data_dict14.loc[i, 'Job Category'] = k
display(job_cat14_list)          
# List to store the variables corresponding to job category for each race.
job_cat14_male_var = []    
job_cat14_female_var = []    

# Extracting the "variable" element from male data dictionary based on the job category matching across alll races
# then adding to a list.
for k in job_cat14_list:
    for i, j in data_dict14.iterrows():
        if (k == j[1]) and (j[3] == 'MALE'):
            job_cat14_male_var.append(j[0])

for k in job_cat14_list:
    for i, j in data_dict14.iterrows():
        if (k == j[1]) and (j[3] == 'FEMALE'):
            job_cat14_female_var.append(j[0])            
            
# Splits the entire list into 10 arrays of size 7 grouping together the job category across all races.
job_cat14_male_varsplit = np.array_split(job_cat14_male_var, 10)
job_cat14_female_varsplit = np.array_split(job_cat14_female_var, 10)

# Initialize a dictionary where I store all the generated dataframes based on the job category for all races.
job_cat14_male_df_dict = {}
job_cat14_female_df_dict = {}

# Fills the dictionary and replaces all empty cells within each dataframe with an NaN value.
n = 0
for entry in job_cat14_list:
    job_cat14_male_df_dict[entry] = df14[list(job_cat14_male_varsplit[n])]
    #display(entry)
    n += 1
    
n = 0    
for entry in job_cat14_list:
    job_cat14_female_df_dict[entry] = df14[list(job_cat14_female_varsplit[n])]
    #display(entry)
    n += 1
    
q1_1_14_tup_male = []
q1_1_14_tup_female = []

# Returns a list of States from the original dataframe.
df14_idx = sorted(list(set(df14.index.get_level_values(0))))

# Create a tuple with "state, key, sum" then convert into dataframe.
for key, value in job_cat14_male_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df14_idx:
        q1_1_14_tup_male.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

for key, value in job_cat14_female_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df14_idx:
        q1_1_14_tup_female.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

# Create dataframe based on tuple.
q1_1_14_male = pd.DataFrame(q1_1_14_tup_male, columns = ['STATE_LABEL', 'Job Category', 'Male Count'])
q1_1_14_male = q1_1_14_male.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_14_female = pd.DataFrame(q1_1_14_tup_female, columns = ['STATE_LABEL', 'Job Category', 'Female Count'])
q1_1_14_female = q1_1_14_female.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_14 = pd.merge(q1_1_14_male, q1_1_14_female, on = ['STATE_LABEL', 'Job Category'])     
q1_1_14 = q1_1_14.set_index(['STATE_LABEL', 'Job Category'])
q1_1_14['Total Count'] = q1_1_14.sum(axis = 1)
q1_1_14['% Men'] = q1_1_14['Male Count']/q1_1_14['Total Count']
q1_1_14['% Women'] = q1_1_14['Female Count']/q1_1_14['Total Count']

#display_side_by_side([q1_1_14_male, q1_1_14_female], ['Male Count in Each Job Category', 'Female Count in Each Job Category'])
display(q1_1_14.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))
#display(job_cat14_male_varsplit)
display(job_cat14_male_varsplit)
#display(job_gen14_male.style.set_caption('Job Categories by Gender'))

### Question 2.0.

In [ ]:
race_ind14_var = []

for k in race_list:
    for i, j in data_dict14.iterrows():
        if (j[1] != '10-TOTAL') and (j[2] == k) and (j[3] == 'FEMALE' or j[3] == 'MALE'):
            race_ind14_var.append(j[0])
            
race_ind14_varsplit = np.array_split(race_ind14_var, 7)

race_ind14_df_dict = {}

q2_0_14 = pd.DataFrame(index = df14.index)

n = 0
for race in race_list:
    race_ind14_df_dict[race] = df14[list(race_ind14_varsplit[n])]
    n += 1

for key, value in race_ind14_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum by Race'] = value.sum(axis = 1)
    #display(key, value)
    q2_0_14[key] = value['Sum by Race']
    
q2_0_14['NAC3 Code Description'] = df14['NAC3_LABEL']  
q2_0_14_col_header = ['NAC3 Code Description', 'AM-INDIAN-ALK', 'ASIAN', 'BLACK', 'HAWAIIAN', 'HISPANIC', 'TWO OR MORE RACES', 'WHITE']
q2_0_14 = q2_0_14[q2_0_14_col_header]
q2_0_14 = q2_0_14.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })

q2_0_14['Total Count'] = q2_0_14.sum(axis = 1)
q2_0_14['% American Indian'] = q2_0_14['AM-INDIAN-ALK Count']/q2_0_14['Total Count']
q2_0_14['% Asian'] = q2_0_14['ASIAN Count']/q2_0_14['Total Count']
q2_0_14['% Black'] = q2_0_14['BLACK Count']/q2_0_14['Total Count']
q2_0_14['% Hawaiian'] = q2_0_14['HAWAIIAN Count']/q2_0_14['Total Count']
q2_0_14['% Hispanic'] = q2_0_14['HISPANIC Count']/q2_0_14['Total Count']
q2_0_14['% Two or More Races'] = q2_0_14['TWO OR MORE RACES Count']/q2_0_14['Total Count']
q2_0_14['% White'] = q2_0_14['WHITE Count']/q2_0_14['Total Count']

display(q2_0_14.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))
     


### Question 2.1.

In [ ]:
job_race14_list = []
job_race14_df_dict = {}
job_race14_key = []
job_race14_key_unq = []
job_race14_var = []

df14_idx = sorted(list(set(df14.index.get_level_values(0))))

q2_1_14_tup = []
 
n = 0
for race in race_list:
    for job in job_cat14_list:
        for i, j in data_dict14.iterrows():
            if (j[1] == job) and (j[2] == race) and (j[3] == 'MALE' or j[3] == 'FEMALE'):
                job_race14_list.append(j[0])
                job_race14_key.append((n, race + ', ' + job))
                n += 1

job_race14_arr = np.array_split(job_race14_list, 70)

for arr in job_race14_arr:
    job_race14_var.append(arr.tolist())

for i in job_race14_key:
    if i[0] % 2 == 0:
        job_race14_key_unq.append(i[1])
        
kv = list(zip(job_race14_key_unq, job_race14_var))

for i, j in kv:
    job_race14_df_dict[i] = df14[j]
    
for key, value in job_race14_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum of Race by Job Category'] = value.sum(axis = 1)
    #display(key, value)
    for state in df14_idx:
        q2_1_14_tup.append(tuple((state, key.split(',')[0],key.split(',')[1].strip(), value.loc[state]['Sum of Race by Job Category'].sum())))
        
# Use the multilevel index from question 1.1 and column headers to be the race list to set up empty dataframe. Iterate through the q2_1_14_tup list to check what race then use
# q2_1_14.loc[state, job category][race] = q2_1_14_tup[3]
q2_1_14 = pd.DataFrame(index = q1_1_14.index, columns = race_list)  

for tup in q2_1_14_tup:
    q2_1_14.loc[tup[0], tup[2]][tup[1]] = tup[3]
    
q2_1_14 = q2_1_14.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })
    
q2_1_14['Total Count'] = q2_1_14.sum(axis = 1)
q2_1_14['% American Indian'] = q2_1_14['AM-INDIAN-ALK Count']/q2_1_14['Total Count']
q2_1_14['% Asian'] = q2_1_14['ASIAN Count']/q2_1_14['Total Count']
q2_1_14['% Black'] = q2_1_14['BLACK Count']/q2_1_14['Total Count']
q2_1_14['% Hawaiian'] = q2_1_14['HAWAIIAN Count']/q2_1_14['Total Count']
q2_1_14['% Hispanic'] = q2_1_14['HISPANIC Count']/q2_1_14['Total Count']
q2_1_14['% Two or More Races'] = q2_1_14['TWO OR MORE RACES Count']/q2_1_14['Total Count']
q2_1_14['% White'] = q2_1_14['WHITE Count']/q2_1_14['Total Count']


display(q2_1_14.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))


## YEAR 13 

In [ ]:
with open('year13_state_nac3.txt') as csvfile:
    data13 = csv.reader(csvfile, delimiter = ';')
    df13 = pd.DataFrame(data13)

header13 = df13.iloc[0]
df13 = df13.iloc[1:]
df13.columns = header13
df13 = df13.set_index(['STATE_LABEL', 'NAC3'])
df13 = df13.replace([None], np.nan)

data_dict13 = pd.read_excel(io = 'data_dictionary-2013.xls', sheet_name = 'state_NAICS3')
data_dict13 = data_dict13.drop(data_dict13.columns[[0, 3, 4]], axis = 1)
data_dict13 = data_dict13.iloc[2:598]
data_dcit13 = data_dict13.reset_index(drop = True, inplace = True)
data_dict13.columns = ['Variable', 'Label']
data_dict13['Job Category'], data_dict13['Race'], data_dict13['Gender'] = data_dict13['Label'].replace('[0-9.-]', '').str.split('/', 2).str
data_dict13 = data_dict13.drop('Label', axis = 1)

data_dict13['Race'] = data_dict13['Race'].replace('Hawaiia', 'HAWAIIAN')
data_dict13['Race'] = data_dict13['Race'].replace(dict.fromkeys(['Two Races', 'Two More Races'], 'TWO OR MORE RACES'))

lat_df13 = data_dict13[data_dict13['Race'].str.contains('HISPANIC', na = False)]
asn_df13 = data_dict13[data_dict13['Race'].str.contains('ASIAN|Asian', na = False)]

asn_errcor13 = asn_df13[(asn_df13['Race'] != 'ASIAN')].copy()
lat_errcor13 = lat_df13[(lat_df13['Race'] != 'HISPANIC')].copy()

asn_errcor13['Race'] = 'ASIAN'
lat_errcor13['Race'] = 'HISPANIC'

lat_errcor13.loc[122, 'Gender'] = 'FEMALE'

# Correcting the dataframe for each race category. This method is useful if there are many entries with the same mistake.
asn_errcor13['Race'] = 'ASIAN'
lat_errcor13['Race'] = 'HISPANIC'
lat_errcor13.loc[122, 'Gender'] = 'FEMALE'

# Updating the data dictionary to be consistent with the preferred variable names for race using original dataframe indices to match.
# Instead of updating each one by row and column, I update the original dataframe with a dataframe comprised of corrected entries.
data_dict13.update(asn_errcor13, overwrite = True)
data_dict13.update(lat_errcor13, overwrite = True)

gen_errcor13 = data_dict13[(data_dict14['Gender'] != 'MALE') & (data_dict13['Gender'] != 'FEMALE') & (data_dict13['Gender'].notnull()) ]
gen_errcor13 = gen_errcor13[~(gen_errcor13['Gender'].str.contains('MINORITY') | gen_errcor13['Gender'].str.contains('PERCENT'))]

data_dict13.loc[144, 'Gender'] = 'MALE'
data_dict13.loc[155, 'Gender'] = 'FEMALE'
data_dict13.loc[354, 'Gender'] = 'MALE(PERCENT)'

#display(gen_errcor13.style.set_caption('Gender Correction'))
#display_side_by_side([asn_errcor13, lat_errcor13], ['Dataframe Error Correction Asian','Dataframe Error Correction Hispanic'])
#display_side_by_side([asn_df13, lat_df13], ['Dataframe Only Asian Race', 'Dataframe Only Hispanic Race'])
display(data_dict13.style.set_caption('2013 Data Dictionary'))
display(df13)

### Question 1.0. 

In [ ]:
data_dict13_male = data_dict13[(data_dict13['Gender'] == 'MALE') & (data_dict13['Job Category'] != '10-TOTAL' )]
data_dict13_female = data_dict13[(data_dict13['Gender'] == 'FEMALE') & (data_dict13['Job Category'] != '10-TOTAL' )]

men13 = data_dict13_male['Variable'].to_list()
women13 = data_dict13_female['Variable'].to_list()

df13_ind_men = df13[men13]
df13_ind_women = df13[women13]

df13_ind_men = df13_ind_men.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df13_ind_men["Men's Count"] = df13_ind_men.sum(axis = 1)

df13_ind_women = df13_ind_women.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df13_ind_women["Women's Count"] = df13_ind_women.sum(axis = 1)

q1_0_13 = pd.DataFrame(index = df13.index)
q1_0_13["Male Count"] = df13_ind_men["Men's Count"]
q1_0_13["Female Count"] = df13_ind_women["Women's Count"]
q1_0_13['Total Count'] = q1_0_13.sum(axis = 1)
q1_0_13['% Men'] = (q1_0_13['Male Count']/q1_0_13['Total Count'])
q1_0_13['% Women'] = (q1_0_13['Female Count']/q1_0_13['Total Count'])
q1_0_13['NAC3 Code Description'] = df13['NAC3_LABEL']
col_head13 = ['NAC3 Code Description', 'Male Count', 'Female Count', 'Total Count', '% Men', '% Women']
q1_0_13 = q1_0_13[col_head13]

#display(len(q1_0_13.groupby(level = 0)))
display(q1_0_13.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))

#display(df13_ind_men, df13_ind_women)

### Question 1.1.

In [ ]:
# Returns the 10 job categories in the dataset.
job_cat13_list = data_dict13.loc[1:11, 'Job Category']
job_cat13_list =job_cat13_list[job_cat13_list != '10-TOTAL'].to_list()

# Corrects the slight mismatch of job categories.
for i, j in data_dict13.iterrows():
    for k in job_cat13_list:
        if j[1][:2] == k[:2]:
            data_dict13.loc[i, 'Job Category'] = k
            
# List to store the variables corresponding to job category for each race.
job_cat13_male_var = []    
job_cat13_female_var = []    

# Extracting the "variable" element from male data dictionary based on the job category matching across alll races
# then adding to a list.
for k in job_cat13_list:
    for i, j in data_dict13.iterrows():
        if (k == j[1]) and (j[3] == 'MALE'):
            job_cat13_male_var.append(j[0])

for k in job_cat13_list:
    for i, j in data_dict13.iterrows():
        if (k == j[1]) and (j[3] == 'FEMALE'):
            job_cat13_female_var.append(j[0])            
          
# Splits the entire list into 10 arrays of size 7 grouping together the job category across all races.
job_cat13_male_varsplit = np.array_split(job_cat13_male_var, 10)
job_cat13_female_varsplit = np.array_split(job_cat13_female_var, 10)

# Initialize a dictionary where I store all the generated dataframes based on the job category for all races.
job_cat13_male_df_dict = {}
job_cat13_female_df_dict = {}

# Fills the dictionary and replaces all empty cells within each dataframe with an NaN value.
n = 0
for entry in job_cat13_list:
    job_cat13_male_df_dict[entry] = df13[list(job_cat13_male_varsplit[n])]
    #display(entry)
    n += 1
    
n = 0    
for entry in job_cat13_list:
    job_cat13_female_df_dict[entry] = df13[list(job_cat13_female_varsplit[n])]
    #display(entry)
    n += 1
    
q1_1_13_tup_male = []
q1_1_13_tup_female = []

# Returns a list of States from the original dataframe.
df13_idx = sorted(list(set(df13.index.get_level_values(0))))

# Create a tuple with "state, key, sum" then convert into dataframe.
for key, value in job_cat13_male_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df13_idx:
        q1_1_13_tup_male.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

for key, value in job_cat13_female_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df13_idx:
        q1_1_13_tup_female.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

# Create dataframe based on tuple.
q1_1_13_male = pd.DataFrame(q1_1_13_tup_male, columns = ['STATE_LABEL', 'Job Category', 'Male Count'])
q1_1_13_male = q1_1_13_male.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_13_female = pd.DataFrame(q1_1_13_tup_female, columns = ['STATE_LABEL', 'Job Category', 'Female Count'])
q1_1_13_female = q1_1_13_female.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_13 = pd.merge(q1_1_13_male, q1_1_13_female, on = ['STATE_LABEL', 'Job Category'])     
q1_1_13 = q1_1_13.set_index(['STATE_LABEL', 'Job Category'])
q1_1_13['Total Count'] = q1_1_13.sum(axis = 1)
q1_1_13['% Men'] = q1_1_13['Male Count']/q1_1_13['Total Count']
q1_1_13['% Women'] = q1_1_13['Female Count']/q1_1_13['Total Count']

#display_side_by_side([q1_1_13_male, q1_1_13_female], ['Male Count in Each Job Category', 'Female Count in Each Job Category'])
display(q1_1_13.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))
#display(job_cat13_male_varsplit)
display(job_cat13_male_varsplit)
#display(job_gen13_male.style.set_caption('Job Categories by Gender'))

### Question 2.0.

In [ ]:
race_ind13_var = []

for k in race_list:
    for i, j in data_dict13.iterrows():
        if (j[1] != '10-TOTAL') and (j[2] == k) and (j[3] == 'FEMALE' or j[3] == 'MALE'):
            race_ind13_var.append(j[0])
            
race_ind13_varsplit = np.array_split(race_ind13_var, 7)

race_ind13_df_dict = {}

q2_0_13 = pd.DataFrame(index = df13.index)

n = 0
for race in race_list:
    race_ind13_df_dict[race] = df13[list(race_ind13_varsplit[n])]
    n += 1

for key, value in race_ind13_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum by Race'] = value.sum(axis = 1)
    #display(key, value)
    q2_0_13[key] = value['Sum by Race']
    
q2_0_13['NAC3 Code Description'] = df13['NAC3_LABEL']  
q2_0_13_col_header = ['NAC3 Code Description', 'AM-INDIAN-ALK', 'ASIAN', 'BLACK', 'HAWAIIAN', 'HISPANIC', 'TWO OR MORE RACES', 'WHITE']
q2_0_13 = q2_0_13[q2_0_13_col_header]
q2_0_13 = q2_0_13.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })

q2_0_13['Total Count'] = q2_0_13.sum(axis = 1)
q2_0_13['% American Indian'] = q2_0_13['AM-INDIAN-ALK Count']/q2_0_13['Total Count']
q2_0_13['% Asian'] = q2_0_13['ASIAN Count']/q2_0_13['Total Count']
q2_0_13['% Black'] = q2_0_13['BLACK Count']/q2_0_13['Total Count']
q2_0_13['% Hawaiian'] = q2_0_13['HAWAIIAN Count']/q2_0_13['Total Count']
q2_0_13['% Hispanic'] = q2_0_13['HISPANIC Count']/q2_0_13['Total Count']
q2_0_13['% Two or More Races'] = q2_0_13['TWO OR MORE RACES Count']/q2_0_13['Total Count']
q2_0_13['% White'] = q2_0_13['WHITE Count']/q2_0_13['Total Count']

display(q2_0_13.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))
     


### Question 2.1.

In [ ]:
job_race13_list = []
job_race13_df_dict = {}
job_race13_key = []
job_race13_key_unq = []
job_race13_var = []

df13_idx = sorted(list(set(df13.index.get_level_values(0))))

q2_1_13_tup = []
 
n = 0
for race in race_list:
    for job in job_cat13_list:
        for i, j in data_dict13.iterrows():
            if (j[1] == job) and (j[2] == race) and (j[3] == 'MALE' or j[3] == 'FEMALE'):
                job_race13_list.append(j[0])
                job_race13_key.append((n, race + ', ' + job))
                n += 1

job_race13_arr = np.array_split(job_race13_list, 70)

for arr in job_race13_arr:
    job_race13_var.append(arr.tolist())

for i in job_race13_key:
    if i[0] % 2 == 0:
        job_race13_key_unq.append(i[1])
        
kv = list(zip(job_race13_key_unq, job_race13_var))

for i, j in kv:
    job_race13_df_dict[i] = df13[j]
    
for key, value in job_race13_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum of Race by Job Category'] = value.sum(axis = 1)
    #display(key, value)
    for state in df13_idx:
        q2_1_13_tup.append(tuple((state, key.split(',')[0],key.split(',')[1].strip(), value.loc[state]['Sum of Race by Job Category'].sum())))
        
# Use the multilevel index from question 1.1 and column headers to be the race list to set up empty dataframe. Iterate through the q2_1_13_tup list to check what race then use
# q2_1_13.loc[state, job category][race] = q2_1_13_tup[3]
q2_1_13 = pd.DataFrame(index = q1_1_13.index, columns = race_list)  

for tup in q2_1_13_tup:
    q2_1_13.loc[tup[0], tup[2]][tup[1]] = tup[3]
    
q2_1_13 = q2_1_13.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })
    
q2_1_13['Total Count'] = q2_1_13.sum(axis = 1)
q2_1_13['% American Indian'] = q2_1_13['AM-INDIAN-ALK Count']/q2_1_13['Total Count']
q2_1_13['% Asian'] = q2_1_13['ASIAN Count']/q2_1_13['Total Count']
q2_1_13['% Black'] = q2_1_13['BLACK Count']/q2_1_13['Total Count']
q2_1_13['% Hawaiian'] = q2_1_13['HAWAIIAN Count']/q2_1_13['Total Count']
q2_1_13['% Hispanic'] = q2_1_13['HISPANIC Count']/q2_1_13['Total Count']
q2_1_13['% Two or More Races'] = q2_1_13['TWO OR MORE RACES Count']/q2_1_13['Total Count']
q2_1_13['% White'] = q2_1_13['WHITE Count']/q2_1_13['Total Count']


display(q2_1_13.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))


## YEAR 12 

In [ ]:
with open('year12_state_nac3.txt') as csvfile:
    data12 = csv.reader(csvfile, delimiter = ';')
    df12 = pd.DataFrame(data12)

header12 = df12.iloc[0]
df12 = df12.iloc[1:]
df12.columns = header12
df12 = df12.drop(df12.columns[2], axis = 1)
df12['NAC3'] = df12['NAC3_Label'].map(naics_dict)
df12 = df12.set_index(['STATE_LABEL', 'NAC3'])
df12 = df12.replace([None], np.nan)
df12 = df12.sort_index(level = [0, 1])

'''
nan_rows = df12[df12['NAC3'].isnull()] # Returns a dataframe made up of rows where there is an NaN value in the "NAC3" column.
nac3_list = nan_rows['NAC3_Label'].tolist() # Takes the data values under the "NAC3_Label" and stores it into a list.
nac3_list = list(set(nac3_list)) # Invoking the .set() function removes duplicated entries then converting the set into a list.
display(nan_rows.head(50)) # Displays 50 rows of the dataframe that consists of data where there was an NaN value in the "NAC3" column.

display(nac3_list)
'''
data_dict12 = pd.read_excel(io = 'data_dictionary-2013.xls', sheet_name = 'state_NAICS3')
data_dict12 = data_dict12.drop(data_dict12.columns[[0, 3, 4]], axis = 1)
data_dict12 = data_dict12.iloc[2:598]
data_dcit12 = data_dict12.reset_index(drop = True, inplace = True)
data_dict12.columns = ['Variable', 'Label']
data_dict12['Job Category'], data_dict12['Race'], data_dict12['Gender'] = data_dict12['Label'].replace('[0-9.-]', '').str.split('/', 2).str
data_dict12 = data_dict12.drop('Label', axis = 1)

data_dict12['Race'] = data_dict12['Race'].replace('Hawaiia', 'HAWAIIAN')
data_dict12['Race'] = data_dict12['Race'].replace(dict.fromkeys(['Two Races', 'Two More Races'], 'TWO OR MORE RACES'))

lat_df12 = data_dict12[data_dict12['Race'].str.contains('HISPANIC', na = False)]
asn_df12 = data_dict12[data_dict12['Race'].str.contains('ASIAN|Asian', na = False)]

asn_errcor12 = asn_df12[(asn_df12['Race'] != 'ASIAN')].copy()
lat_errcor12 = lat_df12[(lat_df12['Race'] != 'HISPANIC')].copy()

asn_errcor12['Race'] = 'ASIAN'
lat_errcor12['Race'] = 'HISPANIC'

lat_errcor12.loc[122, 'Gender'] = 'FEMALE'

# Updating the data dictionary to be consistent with the preferred variable names for race using original dataframe indices to match.
# Instead of updating each one by row and column, I update the original dataframe with a dataframe comprised of corrected entries.
data_dict12.update(asn_errcor12, overwrite = True)
data_dict12.update(lat_errcor12, overwrite = True)

gen_errcor12 = data_dict12[(data_dict12['Gender'] != 'MALE') & (data_dict12['Gender'] != 'FEMALE') & (data_dict12['Gender'].notnull()) ]
gen_errcor12 = gen_errcor12[~(gen_errcor12['Gender'].str.contains('MINORITY') | gen_errcor12['Gender'].str.contains('PERCENT'))]

data_dict12.loc[144, 'Gender'] = 'MALE'
data_dict12.loc[155, 'Gender'] = 'FEMALE'
data_dict12.loc[354, 'Gender'] = 'MALE(PERCENT)'

#display(gen_errcor12.style.set_caption('Gender Correction'))
#display_side_by_side([asn_errcor12, lat_errcor12], ['Dataframe Error Correction Asian','Dataframe Error Correction Hispanic'])
#display_side_by_side([asn_df12, lat_df12], ['Dataframe Only Asian Race', 'Dataframe Only Hispanic Race'])
display(data_dict12.style.set_caption('2012 Data Dictionary'))
display(df12)

### Question 1.0. 

In [ ]:
data_dict12_male = data_dict12[(data_dict12['Gender'] == 'MALE') & (data_dict12['Job Category'] != '10-TOTAL' )]
data_dict12_female = data_dict12[(data_dict12['Gender'] == 'FEMALE') & (data_dict12['Job Category'] != '10-TOTAL' )]

men12 = data_dict12_male['Variable'].to_list()
women12 = data_dict12_female['Variable'].to_list()

df12_ind_men = df12[men12]
df12_ind_women = df12[women12]

df12_ind_men = df12_ind_men.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df12_ind_men["Men's Count"] = df12_ind_men.sum(axis = 1)

df12_ind_women = df12_ind_women.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df12_ind_women["Women's Count"] = df12_ind_women.sum(axis = 1)

q1_0_12 = pd.DataFrame(index = df12.index)
q1_0_12["Male Count"] = df12_ind_men["Men's Count"]
q1_0_12["Female Count"] = df12_ind_women["Women's Count"]
q1_0_12['Total Count'] = q1_0_12.sum(axis = 1)
q1_0_12['% Men'] = (q1_0_12['Male Count']/q1_0_12['Total Count'])
q1_0_12['% Women'] = (q1_0_12['Female Count']/q1_0_12['Total Count'])
q1_0_12['NAC3 Code Description'] = df12['NAC3_Label']
col_head12 = ['NAC3 Code Description', 'Male Count', 'Female Count', 'Total Count', '% Men', '% Women']
q1_0_12 = q1_0_12[col_head12]

#display(len(q1_0_12.groupby(level = 0)))
display(q1_0_12.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))

#display(df12_ind_men, df12_ind_women)

### Question 1.1.

In [ ]:
# Returns the 10 job categories in the dataset.
job_cat12_list = data_dict12.loc[1:11, 'Job Category']
job_cat12_list = job_cat12_list[job_cat12_list != '10-TOTAL'].to_list()

# Corrects the slight mismatch of job categories.
for i, j in data_dict12.iterrows():
    for k in job_cat12_list:
        if j[1][:2] == k[:2]:
            data_dict12.loc[i, 'Job Category'] = k
            
# List to store the variables corresponding to job category for each race.
job_cat12_male_var = []    
job_cat12_female_var = []    

# Extracting the "variable" element from male data dictionary based on the job category matching across alll races
# then adding to a list.
for k in job_cat12_list:
    for i, j in data_dict12.iterrows():
        if (k == j[1]) and (j[3] == 'MALE'):
            job_cat12_male_var.append(j[0])

for k in job_cat12_list:
    for i, j in data_dict12.iterrows():
        if (k == j[1]) and (j[3] == 'FEMALE'):
            job_cat12_female_var.append(j[0])            

# Splits the entire list into 10 arrays of size 7 grouping together the job category across all races.
job_cat12_male_varsplit = np.array_split(job_cat12_male_var, 10)
job_cat12_female_varsplit = np.array_split(job_cat12_female_var, 10)

# Initialize a dictionary where I store all the generated dataframes based on the job category for all races.
job_cat12_male_df_dict = {}
job_cat12_female_df_dict = {}

# Fills the dictionary and replaces all empty cells within each dataframe with an NaN value.
n = 0
for entry in job_cat12_list:
    job_cat12_male_df_dict[entry] = df12[list(job_cat12_male_varsplit[n])]
    #display(entry)
    n += 1
    
n = 0    
for entry in job_cat12_list:
    job_cat12_female_df_dict[entry] = df12[list(job_cat12_female_varsplit[n])]
    #display(entry)
    n += 1
    
q1_1_12_tup_male = []
q1_1_12_tup_female = []

# Returns a list of States from the original dataframe.
df12_idx = sorted(list(set(df12.index.get_level_values(0))))

# Create a tuple with "state, key, sum" then convert into dataframe.
for key, value in job_cat12_male_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df12_idx:
        q1_1_12_tup_male.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

for key, value in job_cat12_female_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df12_idx:
        q1_1_12_tup_female.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

# Create dataframe based on tuple.
q1_1_12_male = pd.DataFrame(q1_1_12_tup_male, columns = ['STATE_LABEL', 'Job Category', 'Male Count'])
q1_1_12_male = q1_1_12_male.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_12_female = pd.DataFrame(q1_1_12_tup_female, columns = ['STATE_LABEL', 'Job Category', 'Female Count'])
q1_1_12_female = q1_1_12_female.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_12 = pd.merge(q1_1_12_male, q1_1_12_female, on = ['STATE_LABEL', 'Job Category'])     
q1_1_12 = q1_1_12.set_index(['STATE_LABEL', 'Job Category'])
q1_1_12['Total Count'] = q1_1_12.sum(axis = 1)
q1_1_12['% Men'] = q1_1_12['Male Count']/q1_1_12['Total Count']
q1_1_12['% Women'] = q1_1_12['Female Count']/q1_1_12['Total Count']

#display_side_by_side([q1_1_12_male, q1_1_12_female], ['Male Count in Each Job Category', 'Female Count in Each Job Category'])
display(q1_1_12.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))
#display(job_cat12_male_varsplit)
display(job_cat12_male_varsplit)
#display(job_gen12_male.style.set_caption('Job Categories by Gender'))

### Question 2.0.

In [ ]:
race_ind12_var = []

for k in race_list:
    for i, j in data_dict12.iterrows():
        if (j[1] != '10-TOTAL') and (j[2] == k) and (j[3] == 'FEMALE' or j[3] == 'MALE'):
            race_ind12_var.append(j[0])
            
race_ind12_varsplit = np.array_split(race_ind12_var, 7)

race_ind12_df_dict = {}

q2_0_12 = pd.DataFrame(index = df12.index)

n = 0
for race in race_list:
    race_ind12_df_dict[race] = df12[list(race_ind12_varsplit[n])]
    n += 1

for key, value in race_ind12_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum by Race'] = value.sum(axis = 1)
    #display(key, value)
    q2_0_12[key] = value['Sum by Race']
    
q2_0_12['NAC3 Code Description'] = df12['NAC3_Label']  
q2_0_12_col_header = ['NAC3 Code Description', 'AM-INDIAN-ALK', 'ASIAN', 'BLACK', 'HAWAIIAN', 'HISPANIC', 'TWO OR MORE RACES', 'WHITE']
q2_0_12 = q2_0_12[q2_0_12_col_header]
q2_0_12 = q2_0_12.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })

q2_0_12['Total Count'] = q2_0_12.sum(axis = 1)
q2_0_12['% American Indian'] = q2_0_12['AM-INDIAN-ALK Count']/q2_0_12['Total Count']
q2_0_12['% Asian'] = q2_0_12['ASIAN Count']/q2_0_12['Total Count']
q2_0_12['% Black'] = q2_0_12['BLACK Count']/q2_0_12['Total Count']
q2_0_12['% Hawaiian'] = q2_0_12['HAWAIIAN Count']/q2_0_12['Total Count']
q2_0_12['% Hispanic'] = q2_0_12['HISPANIC Count']/q2_0_12['Total Count']
q2_0_12['% Two or More Races'] = q2_0_12['TWO OR MORE RACES Count']/q2_0_12['Total Count']
q2_0_12['% White'] = q2_0_12['WHITE Count']/q2_0_12['Total Count']

display(q2_0_12.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))
     


### Question 2.1.

In [ ]:
job_race12_list = []
job_race12_df_dict = {}
job_race12_key = []
job_race12_key_unq = []
job_race12_var = []

df12_idx = sorted(list(set(df12.index.get_level_values(0))))

q2_1_12_tup = []
 
n = 0
for race in race_list:
    for job in job_cat12_list:
        for i, j in data_dict12.iterrows():
            if (j[1] == job) and (j[2] == race) and (j[3] == 'MALE' or j[3] == 'FEMALE'):
                job_race12_list.append(j[0])
                job_race12_key.append((n, race + ', ' + job))
                n += 1

job_race12_arr = np.array_split(job_race12_list, 70)

for arr in job_race12_arr:
    job_race12_var.append(arr.tolist())

for i in job_race12_key:
    if i[0] % 2 == 0:
        job_race12_key_unq.append(i[1])
        
kv = list(zip(job_race12_key_unq, job_race12_var))

for i, j in kv:
    job_race12_df_dict[i] = df12[j]
    
for key, value in job_race12_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum of Race by Job Category'] = value.sum(axis = 1)
    #display(key, value)
    for state in df12_idx:
        q2_1_12_tup.append(tuple((state, key.split(',')[0],key.split(',')[1].strip(), value.loc[state]['Sum of Race by Job Category'].sum())))
        
# Use the multilevel index from question 1.1 and column headers to be the race list to set up empty dataframe. Iterate through the q2_1_12_tup list to check what race then use
# q2_1_12.loc[state, job category][race] = q2_1_12_tup[3]
q2_1_12 = pd.DataFrame(index = q1_1_12.index, columns = race_list)  

for tup in q2_1_12_tup:
    q2_1_12.loc[tup[0], tup[2]][tup[1]] = tup[3]
    
q2_1_12 = q2_1_12.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })
    
q2_1_12['Total Count'] = q2_1_12.sum(axis = 1)
q2_1_12['% American Indian'] = q2_1_12['AM-INDIAN-ALK Count']/q2_1_12['Total Count']
q2_1_12['% Asian'] = q2_1_12['ASIAN Count']/q2_1_12['Total Count']
q2_1_12['% Black'] = q2_1_12['BLACK Count']/q2_1_12['Total Count']
q2_1_12['% Hawaiian'] = q2_1_12['HAWAIIAN Count']/q2_1_12['Total Count']
q2_1_12['% Hispanic'] = q2_1_12['HISPANIC Count']/q2_1_12['Total Count']
q2_1_12['% Two or More Races'] = q2_1_12['TWO OR MORE RACES Count']/q2_1_12['Total Count']
q2_1_12['% White'] = q2_1_12['WHITE Count']/q2_1_12['Total Count']


display(q2_1_12.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))


## YEAR 11 

In [ ]:
with open('year11_state_nac3.txt') as csvfile:
    data11 = csv.reader(csvfile, delimiter = ';')
    df11 = pd.DataFrame(data11)

header11 = df11.iloc[0]
df11 = df11.iloc[1:]
df11.columns = header11
df11['NAC3'] = df11['NAC3_Label'].map(naics_dict)
df11 = df11.set_index(['STATE_LABEL', 'NAC3'])
df11 = df11.drop(['_TYPE_'], axis = 1)
df11 = df11.replace([None], np.nan)
df11 = df11.sort_index(level = [0, 1], ascending = True)

data_dict11 = pd.read_excel(io = 'data_dictionary-2013.xls', sheet_name = 'state_NAICS3')
data_dict11 = data_dict11.drop(data_dict11.columns[[0, 3, 4]], axis = 1)
data_dict11 = data_dict11.iloc[2:598]
data_dcit11 = data_dict11.reset_index(drop = True, inplace = True)
data_dict11.columns = ['Variable', 'Label']
data_dict11['Job Category'], data_dict11['Race'], data_dict11['Gender'] = data_dict11['Label'].replace('[0-9.-]', '').str.split('/', 2).str
data_dict11 = data_dict11.drop('Label', axis = 1)

data_dict11['Race'] = data_dict11['Race'].replace('Hawaiia', 'HAWAIIAN')
data_dict11['Race'] = data_dict11['Race'].replace(dict.fromkeys(['Two Races', 'Two More Races'], 'TWO OR MORE RACES'))

lat_df11 = data_dict11[data_dict11['Race'].str.contains('HISPANIC', na = False)]
asn_df11 = data_dict11[data_dict11['Race'].str.contains('ASIAN|Asian', na = False)]

asn_errcor11 = asn_df11[(asn_df11['Race'] != 'ASIAN')].copy()
lat_errcor11 = lat_df11[(lat_df11['Race'] != 'HISPANIC')].copy()

asn_errcor11['Race'] = 'ASIAN'
lat_errcor11['Race'] = 'HISPANIC'

lat_errcor11.loc[122, 'Gender'] = 'FEMALE'

# Updating the data dictionary to be consistent with the preferred variable names for race using original dataframe indices to match.
# Instead of updating each one by row and column, I update the original dataframe with a dataframe comprised of corrected entries.
data_dict11.update(asn_errcor11, overwrite = True)
data_dict11.update(lat_errcor11, overwrite = True)

gen_errcor11 = data_dict11[(data_dict11['Gender'] != 'MALE') & (data_dict11['Gender'] != 'FEMALE') & (data_dict11['Gender'].notnull()) ]
gen_errcor11 = gen_errcor11[~(gen_errcor11['Gender'].str.contains('MINORITY') | gen_errcor11['Gender'].str.contains('PERCENT'))]

data_dict11.loc[144, 'Gender'] = 'MALE'
data_dict11.loc[155, 'Gender'] = 'FEMALE'
data_dict11.loc[354, 'Gender'] = 'MALE(PERCENT)'

#display(gen_errcor11.style.set_caption('Gender Correction'))
#display_side_by_side([asn_errcor11, lat_errcor11], ['Dataframe Error Correction Asian','Dataframe Error Correction Hispanic'])
#display_side_by_side([asn_df11, lat_df11], ['Dataframe Only Asian Race', 'Dataframe Only Hispanic Race'])
display(data_dict11.style.set_caption('2011 Data Dictionary'))
display(df11)

### Question 1.0. 

In [ ]:
data_dict11_male = data_dict11[(data_dict11['Gender'] == 'MALE') & (data_dict11['Job Category'] != '10-TOTAL' )]
data_dict11_female = data_dict11[(data_dict11['Gender'] == 'FEMALE') & (data_dict11['Job Category'] != '10-TOTAL' )]

men11 = data_dict11_male['Variable'].to_list()
women11 = data_dict11_female['Variable'].to_list()

df11_ind_men = df11[men11]
df11_ind_women = df11[women11]

df11_ind_men = df11_ind_men.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df11_ind_men["Men's Count"] = df11_ind_men.sum(axis = 1)

df11_ind_women = df11_ind_women.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df11_ind_women["Women's Count"] = df11_ind_women.sum(axis = 1)

q1_0_11 = pd.DataFrame(index = df11.index)
q1_0_11["Male Count"] = df11_ind_men["Men's Count"]
q1_0_11["Female Count"] = df11_ind_women["Women's Count"]
q1_0_11['Total Count'] = q1_0_11.sum(axis = 1)
q1_0_11['% Men'] = (q1_0_11['Male Count']/q1_0_11['Total Count'])
q1_0_11['% Women'] = (q1_0_11['Female Count']/q1_0_11['Total Count'])
q1_0_11['NAC3 Code Description'] = df11['NAC3_Label']
col_head11 = ['NAC3 Code Description', 'Male Count', 'Female Count', 'Total Count', '% Men', '% Women']
q1_0_11 = q1_0_11[col_head11]

#display(len(q1_0_11.groupby(level = 0)))
display(q1_0_11.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))
display(q1_0_11.loc[('Minnesota', '541'), ('% Women', 'Total Count')])
#display(df11_ind_men, df11_ind_women)

### Question 1.1.

In [ ]:
# Returns the 10 job categories in the dataset.
job_cat11_list = data_dict11.loc[1:11, 'Job Category']
job_cat11_list = job_cat11_list[job_cat11_list != '10-TOTAL'].to_list()

# Corrects the slight mismatch of job categories.
for i, j in data_dict11.iterrows():
    for k in job_cat11_list:
        if j[1][:2] == k[:2]:
            data_dict11.loc[i, 'Job Category'] = k
            
# List to store the variables corresponding to job category for each race.
job_cat11_male_var = []    
job_cat11_female_var = []    

# Extracting the "variable" element from male data dictionary based on the job category matching across alll races
# then adding to a list.
for k in job_cat11_list:
    for i, j in data_dict11.iterrows():
        if (k == j[1]) and (j[3] == 'MALE'):
            job_cat11_male_var.append(j[0])

for k in job_cat11_list:
    for i, j in data_dict11.iterrows():
        if (k == j[1]) and (j[3] == 'FEMALE'):
            job_cat11_female_var.append(j[0])            
            
# Splits the entire list into 10 arrays of size 7 grouping together the job category across all races.
job_cat11_male_varsplit = np.array_split(job_cat11_male_var, 10)
job_cat11_female_varsplit = np.array_split(job_cat11_female_var, 10)

# Initialize a dictionary where I store all the generated dataframes based on the job category for all races.
job_cat11_male_df_dict = {}
job_cat11_female_df_dict = {}

# Fills the dictionary and replaces all empty cells within each dataframe with an NaN value.
n = 0
for entry in job_cat11_list:
    job_cat11_male_df_dict[entry] = df11[list(job_cat11_male_varsplit[n])]
    #display(entry)
    n += 1
    
n = 0    
for entry in job_cat11_list:
    job_cat11_female_df_dict[entry] = df11[list(job_cat11_female_varsplit[n])]
    #display(entry)
    n += 1
    
q1_1_11_tup_male = []
q1_1_11_tup_female = []

# Returns a list of States from the original dataframe.
df11_idx = sorted(list(set(df11.index.get_level_values(0))))

# Create a tuple with "state, key, sum" then convert into dataframe.
for key, value in job_cat11_male_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df11_idx:
        q1_1_11_tup_male.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

for key, value in job_cat11_female_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df11_idx:
        q1_1_11_tup_female.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

# Create dataframe based on tuple.
q1_1_11_male = pd.DataFrame(q1_1_11_tup_male, columns = ['STATE_LABEL', 'Job Category', 'Male Count'])
q1_1_11_male = q1_1_11_male.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_11_female = pd.DataFrame(q1_1_11_tup_female, columns = ['STATE_LABEL', 'Job Category', 'Female Count'])
q1_1_11_female = q1_1_11_female.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_11 = pd.merge(q1_1_11_male, q1_1_11_female, on = ['STATE_LABEL', 'Job Category'])     
q1_1_11 = q1_1_11.set_index(['STATE_LABEL', 'Job Category'])
q1_1_11['Total Count'] = q1_1_11.sum(axis = 1)
q1_1_11['% Men'] = q1_1_11['Male Count']/q1_1_11['Total Count']
q1_1_11['% Women'] = q1_1_11['Female Count']/q1_1_11['Total Count']

#display_side_by_side([q1_1_11_male, q1_1_11_female], ['Male Count in Each Job Category', 'Female Count in Each Job Category'])
display(q1_1_11.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))
#display(job_cat11_male_varsplit)
display(job_cat11_male_varsplit)
#display(job_gen11_male.style.set_caption('Job Categories by Gender'))

### Question 2.0.

In [ ]:
race_ind11_var = []

for k in race_list:
    for i, j in data_dict11.iterrows():
        if (j[1] != '10-TOTAL') and (j[2] == k) and (j[3] == 'FEMALE' or j[3] == 'MALE'):
            race_ind11_var.append(j[0])
            
race_ind11_varsplit = np.array_split(race_ind11_var, 7)

race_ind11_df_dict = {}

q2_0_11 = pd.DataFrame(index = df11.index)

n = 0
for race in race_list:
    race_ind11_df_dict[race] = df11[list(race_ind11_varsplit[n])]
    n += 1

for key, value in race_ind11_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum by Race'] = value.sum(axis = 1)
    #display(key, value)
    q2_0_11[key] = value['Sum by Race']
    
q2_0_11['NAC3 Code Description'] = df11['NAC3_Label']  
q2_0_11_col_header = ['NAC3 Code Description', 'AM-INDIAN-ALK', 'ASIAN', 'BLACK', 'HAWAIIAN', 'HISPANIC', 'TWO OR MORE RACES', 'WHITE']
q2_0_11 = q2_0_11[q2_0_11_col_header]
q2_0_11 = q2_0_11.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })

q2_0_11['Total Count'] = q2_0_11.sum(axis = 1)
q2_0_11['% American Indian'] = q2_0_11['AM-INDIAN-ALK Count']/q2_0_11['Total Count']
q2_0_11['% Asian'] = q2_0_11['ASIAN Count']/q2_0_11['Total Count']
q2_0_11['% Black'] = q2_0_11['BLACK Count']/q2_0_11['Total Count']
q2_0_11['% Hawaiian'] = q2_0_11['HAWAIIAN Count']/q2_0_11['Total Count']
q2_0_11['% Hispanic'] = q2_0_11['HISPANIC Count']/q2_0_11['Total Count']
q2_0_11['% Two or More Races'] = q2_0_11['TWO OR MORE RACES Count']/q2_0_11['Total Count']
q2_0_11['% White'] = q2_0_11['WHITE Count']/q2_0_11['Total Count']

display(q2_0_11.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))
     


### Question 2.1.

In [ ]:
job_race11_list = []
job_race11_df_dict = {}
job_race11_key = []
job_race11_key_unq = []
job_race11_var = []

df11_idx = sorted(list(set(df11.index.get_level_values(0))))

q2_1_11_tup = []
 
n = 0
for race in race_list:
    for job in job_cat11_list:
        for i, j in data_dict11.iterrows():
            if (j[1] == job) and (j[2] == race) and (j[3] == 'MALE' or j[3] == 'FEMALE'):
                job_race11_list.append(j[0])
                job_race11_key.append((n, race + ', ' + job))
                n += 1

job_race11_arr = np.array_split(job_race11_list, 70)

for arr in job_race11_arr:
    job_race11_var.append(arr.tolist())

for i in job_race11_key:
    if i[0] % 2 == 0:
        job_race11_key_unq.append(i[1])
        
kv = list(zip(job_race11_key_unq, job_race11_var))

for i, j in kv:
    job_race11_df_dict[i] = df11[j]
    
for key, value in job_race11_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum of Race by Job Category'] = value.sum(axis = 1)
    #display(key, value)
    for state in df11_idx:
        q2_1_11_tup.append(tuple((state, key.split(',')[0],key.split(',')[1].strip(), value.loc[state]['Sum of Race by Job Category'].sum())))
        
# Use the multilevel index from question 1.1 and column headers to be the race list to set up empty dataframe. Iterate through the q2_1_11_tup list to check what race then use
# q2_1_11.loc[state, job category][race] = q2_1_11_tup[3]
q2_1_11 = pd.DataFrame(index = q1_1_11.index, columns = race_list)  

for tup in q2_1_11_tup:
    q2_1_11.loc[tup[0], tup[2]][tup[1]] = tup[3]
    
q2_1_11 = q2_1_11.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })
    
q2_1_11['Total Count'] = q2_1_11.sum(axis = 1)
q2_1_11['% American Indian'] = q2_1_11['AM-INDIAN-ALK Count']/q2_1_11['Total Count']
q2_1_11['% Asian'] = q2_1_11['ASIAN Count']/q2_1_11['Total Count']
q2_1_11['% Black'] = q2_1_11['BLACK Count']/q2_1_11['Total Count']
q2_1_11['% Hawaiian'] = q2_1_11['HAWAIIAN Count']/q2_1_11['Total Count']
q2_1_11['% Hispanic'] = q2_1_11['HISPANIC Count']/q2_1_11['Total Count']
q2_1_11['% Two or More Races'] = q2_1_11['TWO OR MORE RACES Count']/q2_1_11['Total Count']
q2_1_11['% White'] = q2_1_11['WHITE Count']/q2_1_11['Total Count']


display(q2_1_11.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))


## YEAR 10

In [ ]:
with open('year10_state_nac3.txt') as csvfile:
    data10 = csv.reader(csvfile, delimiter = ';')
    df10 = pd.DataFrame(data10)

header10 = df10.iloc[0]
df10 = df10.iloc[1:]
df10.columns = header10
df10 = df10.drop(df10.columns[2], axis = 1)
df10['NAC3'] = df10['NAC3_Label'].map(naics_dict)
df10 = df10.set_index(['STATE_LABEL', 'NAC3'])
df10 = df10.replace([None], np.nan)
df10 = df10.sort_index(level = [0, 1])

data_dict10 = pd.read_excel(io = 'data_dictionary 2010.xls', sheet_name = 'state_NAICS3')
data_dict10 = data_dict10.drop(data_dict10.columns[[0, 3, 4]], axis = 1)
data_dict10 = data_dict10.iloc[2:598]
data_dcit10 = data_dict10.reset_index(drop = True, inplace = True)
data_dict10.columns = ['Variable', 'Label']
data_dict10['Job Category'], data_dict10['Race'], data_dict10['Gender'] = data_dict10['Label'].replace('[0-9.-]', '').str.split('/', 2).str
data_dict10 = data_dict10.drop('Label', axis = 1)

data_dict10['Race'] = data_dict10['Race'].replace('Hawaiia', 'HAWAIIAN')
data_dict10['Race'] = data_dict10['Race'].replace(dict.fromkeys(['Two Races', 'Two More Races'], 'TWO OR MORE RACES'))

lat_df10 = data_dict10[data_dict10['Race'].str.contains('HISPANIC', na = False)]
asn_df10 = data_dict10[data_dict10['Race'].str.contains('ASIAN|Asian', na = False)]

asn_errcor10 = asn_df10[(asn_df10['Race'] != 'ASIAN')].copy()
lat_errcor10 = lat_df10[(lat_df10['Race'] != 'HISPANIC')].copy()

asn_errcor10['Race'] = 'ASIAN'
lat_errcor10['Race'] = 'HISPANIC'

lat_errcor10.loc[122, 'Gender'] = 'FEMALE'

# Updating the data dictionary to be consistent with the preferred variable names for race using original dataframe indices to match.
# Instead of updating each one by row and column, I update the original dataframe with a dataframe comprised of corrected entries.
data_dict10.update(asn_errcor10, overwrite = True)
data_dict10.update(lat_errcor10, overwrite = True)

gen_errcor10 = data_dict10[(data_dict10['Gender'] != 'MALE') & (data_dict10['Gender'] != 'FEMALE') & (data_dict10['Gender'].notnull()) ]
gen_errcor10 = gen_errcor10[~(gen_errcor10['Gender'].str.contains('MINORITY') | gen_errcor10['Gender'].str.contains('PERCENT'))]

data_dict10.loc[144, 'Gender'] = 'MALE'
data_dict10.loc[155, 'Gender'] = 'FEMALE'
data_dict10.loc[354, 'Gender'] = 'MALE(PERCENT)'

#display(gen_errcor10.style.set_caption('Gender Correction'))
#display_side_by_side([asn_errcor10, lat_errcor10], ['Dataframe Error Correction Asian','Dataframe Error Correction Hispanic'])
#display_side_by_side([asn_df10, lat_df10], ['Dataframe Only Asian Race', 'Dataframe Only Hispanic Race'])
display(data_dict10.style.set_caption('2010 Data Dictionary'))
display(df10)


### Question 1.0.

In [ ]:
data_dict10_male = data_dict10[(data_dict10['Gender'] == 'MALE') & (data_dict10['Job Category'] != '10-TOTAL' )]
data_dict10_female = data_dict10[(data_dict10['Gender'] == 'FEMALE') & (data_dict10['Job Category'] != '10-TOTAL' )]

men10 = data_dict10_male['Variable'].to_list()
women10 = data_dict10_female['Variable'].to_list()
display(men10)
df10_ind_men = df10[men10]
df10_ind_women = df10[women10]

df10_ind_men = df10_ind_men.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df10_ind_men["Men's Count"] = df10_ind_men.sum(axis = 1)

df10_ind_women = df10_ind_women.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df10_ind_women["Women's Count"] = df10_ind_women.sum(axis = 1)

q1_0_10 = pd.DataFrame(index = df10.index)
q1_0_10["Male Count"] = df10_ind_men["Men's Count"]
q1_0_10["Female Count"] = df10_ind_women["Women's Count"]
q1_0_10['Total Count'] = q1_0_10.sum(axis = 1)
q1_0_10['% Men'] = (q1_0_10['Male Count']/q1_0_10['Total Count'])
q1_0_10['% Women'] = (q1_0_10['Female Count']/q1_0_10['Total Count'])
q1_0_10['NAC3 Code Description'] = df10['NAC3_Label']
col_head10 = ['NAC3 Code Description', 'Male Count', 'Female Count', 'Total Count', '% Men', '% Women']
q1_0_10 = q1_0_10[col_head10]

#display(len(q1_0_10.groupby(level = 0)))
display(q1_0_10.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))
#display(q1_0_10.loc[('Minnesota', '541'), ('% Women', 'Total Count')])
#display(df10_ind_men, df10_ind_women)

### Question 1.1.

In [ ]:
# Returns the 10 job categories in the dataset.
job_cat10_list = data_dict10.loc[1:11, 'Job Category']
job_cat10_list = job_cat10_list[job_cat10_list != '10-TOTAL'].to_list()

# Corrects the slight mismatch of job categories.
for i, j in data_dict10.iterrows():
    for k in job_cat10_list:
        if j[1][:2] == k[:2]:
            data_dict10.loc[i, 'Job Category'] = k
            
# List to store the variables corresponding to job category for each race.
job_cat10_male_var = []    
job_cat10_female_var = []    

# Extracting the "variable" element from male data dictionary based on the job category matching across alll races
# then adding to a list.
for k in job_cat10_list:
    for i, j in data_dict10.iterrows():
        if (k == j[1]) and (j[3] == 'MALE'):
            job_cat10_male_var.append(j[0])

for k in job_cat10_list:
    for i, j in data_dict10.iterrows():
        if (k == j[1]) and (j[3] == 'FEMALE'):
            job_cat10_female_var.append(j[0])            
            
# Splits the entire list into 10 arrays of size 7 grouping together the job category across all races.
job_cat10_male_varsplit = np.array_split(job_cat10_male_var, 10)
job_cat10_female_varsplit = np.array_split(job_cat10_female_var, 10)

# Initialize a dictionary where I store all the generated dataframes based on the job category for all races.
job_cat10_male_df_dict = {}
job_cat10_female_df_dict = {}

# Fills the dictionary and replaces all empty cells within each dataframe with an NaN value.
n = 0
for entry in job_cat10_list:
    job_cat10_male_df_dict[entry] = df10[list(job_cat10_male_varsplit[n])]
    #display(entry)
    n += 1
    
n = 0    
for entry in job_cat10_list:
    job_cat10_female_df_dict[entry] = df10[list(job_cat10_female_varsplit[n])]
    #display(entry)
    n += 1
    
q1_1_10_tup_male = []
q1_1_10_tup_female = []

# Returns a list of States from the original dataframe.
df10_idx = sorted(list(set(df10.index.get_level_values(0))))

# Create a tuple with "state, key, sum" then convert into dataframe.
for key, value in job_cat10_male_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df10_idx:
        q1_1_10_tup_male.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

for key, value in job_cat10_female_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df10_idx:
        q1_1_10_tup_female.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

# Create dataframe based on tuple.
q1_1_10_male = pd.DataFrame(q1_1_10_tup_male, columns = ['STATE_LABEL', 'Job Category', 'Male Count'])
q1_1_10_male = q1_1_10_male.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_10_female = pd.DataFrame(q1_1_10_tup_female, columns = ['STATE_LABEL', 'Job Category', 'Female Count'])
q1_1_10_female = q1_1_10_female.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_10 = pd.merge(q1_1_10_male, q1_1_10_female, on = ['STATE_LABEL', 'Job Category'])     
q1_1_10 = q1_1_10.set_index(['STATE_LABEL', 'Job Category'])
q1_1_10['Total Count'] = q1_1_10.sum(axis = 1)
q1_1_10['% Men'] = q1_1_10['Male Count']/q1_1_10['Total Count']
q1_1_10['% Women'] = q1_1_10['Female Count']/q1_1_10['Total Count']

#display_side_by_side([q1_1_10_male, q1_1_10_female], ['Male Count in Each Job Category', 'Female Count in Each Job Category'])
display(q1_1_10.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))
#display(job_cat10_male_varsplit)
display(job_cat10_male_varsplit)
#display(job_gen10_male.style.set_caption('Job Categories by Gender'))

### Question 2.0.

In [ ]:
race_ind10_var = []

for k in race_list:
    for i, j in data_dict10.iterrows():
        if (j[1] != '10-TOTAL') and (j[2] == k) and (j[3] == 'FEMALE' or j[3] == 'MALE'):
            race_ind10_var.append(j[0])
            
race_ind10_varsplit = np.array_split(race_ind10_var, 7)

race_ind10_df_dict = {}

q2_0_10 = pd.DataFrame(index = df10.index)

n = 0
for race in race_list:
    race_ind10_df_dict[race] = df10[list(race_ind10_varsplit[n])]
    n += 1

for key, value in race_ind10_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum by Race'] = value.sum(axis = 1)
    #display(key, value)
    q2_0_10[key] = value['Sum by Race']
    
q2_0_10['NAC3 Code Description'] = df10['NAC3_Label']  
q2_0_10_col_header = ['NAC3 Code Description', 'AM-INDIAN-ALK', 'ASIAN', 'BLACK', 'HAWAIIAN', 'HISPANIC', 'TWO OR MORE RACES', 'WHITE']
q2_0_10 = q2_0_10[q2_0_10_col_header]
q2_0_10 = q2_0_10.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })

q2_0_10['Total Count'] = q2_0_10.sum(axis = 1)
q2_0_10['% American Indian'] = q2_0_10['AM-INDIAN-ALK Count']/q2_0_10['Total Count']
q2_0_10['% Asian'] = q2_0_10['ASIAN Count']/q2_0_10['Total Count']
q2_0_10['% Black'] = q2_0_10['BLACK Count']/q2_0_10['Total Count']
q2_0_10['% Hawaiian'] = q2_0_10['HAWAIIAN Count']/q2_0_10['Total Count']
q2_0_10['% Hispanic'] = q2_0_10['HISPANIC Count']/q2_0_10['Total Count']
q2_0_10['% Two or More Races'] = q2_0_10['TWO OR MORE RACES Count']/q2_0_10['Total Count']
q2_0_10['% White'] = q2_0_10['WHITE Count']/q2_0_10['Total Count']

display(q2_0_10.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))
     


### Question 2.1.

In [ ]:
job_race10_list = []
job_race10_df_dict = {}
job_race10_key = []
job_race10_key_unq = []
job_race10_var = []

df10_idx = sorted(list(set(df10.index.get_level_values(0))))

q2_1_10_tup = []
 
n = 0
for race in race_list:
    for job in job_cat10_list:
        for i, j in data_dict10.iterrows():
            if (j[1] == job) and (j[2] == race) and (j[3] == 'MALE' or j[3] == 'FEMALE'):
                job_race10_list.append(j[0])
                job_race10_key.append((n, race + ', ' + job))
                n += 1

job_race10_arr = np.array_split(job_race10_list, 70)

for arr in job_race10_arr:
    job_race10_var.append(arr.tolist())

for i in job_race10_key:
    if i[0] % 2 == 0:
        job_race10_key_unq.append(i[1])
        
kv = list(zip(job_race10_key_unq, job_race10_var))

for i, j in kv:
    job_race10_df_dict[i] = df10[j]
    
for key, value in job_race10_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum of Race by Job Category'] = value.sum(axis = 1)
    #display(key, value)
    for state in df10_idx:
        q2_1_10_tup.append(tuple((state, key.split(',')[0],key.split(',')[1].strip(), value.loc[state]['Sum of Race by Job Category'].sum())))
        
# Use the multilevel index from question 1.1 and column headers to be the race list to set up empty dataframe. Iterate through the q2_1_10_tup list to check what race then use
# q2_1_10.loc[state, job category][race] = q2_1_10_tup[3]
q2_1_10 = pd.DataFrame(index = q1_1_10.index, columns = race_list)  

for tup in q2_1_10_tup:
    q2_1_10.loc[tup[0], tup[2]][tup[1]] = tup[3]
    
q2_1_10 = q2_1_10.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })
    
q2_1_10['Total Count'] = q2_1_10.sum(axis = 1)
q2_1_10['% American Indian'] = q2_1_10['AM-INDIAN-ALK Count']/q2_1_10['Total Count']
q2_1_10['% Asian'] = q2_1_10['ASIAN Count']/q2_1_10['Total Count']
q2_1_10['% Black'] = q2_1_10['BLACK Count']/q2_1_10['Total Count']
q2_1_10['% Hawaiian'] = q2_1_10['HAWAIIAN Count']/q2_1_10['Total Count']
q2_1_10['% Hispanic'] = q2_1_10['HISPANIC Count']/q2_1_10['Total Count']
q2_1_10['% Two or More Races'] = q2_1_10['TWO OR MORE RACES Count']/q2_1_10['Total Count']
q2_1_10['% White'] = q2_1_10['WHITE Count']/q2_1_10['Total Count']


display(q2_1_10.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))


## YEAR 09

In [ ]:
with open('year09_state_nac3.txt') as csvfile:
    data09 = csv.reader(csvfile, delimiter = ';')
    df09 = pd.DataFrame(data09)

header09 = df09.iloc[0]
df09 = df09.iloc[1:]
df09.columns = header09
df09 = df09.drop(df09.columns[2], axis = 1)
df09['NAC3'] = df09['NAC3_Label'].map(naics_dict)
df09 = df09.set_index(['STATE_LABEL', 'NAC3'])
df09 = df09.replace([None], np.nan)
df09 = df09.sort_index(level = [0, 1])

data_dict09 = pd.read_excel(io = 'data_dictionary NAICS 2009.xls', sheet_name = 'state_NAICS3')
data_dict09 = data_dict09.drop(data_dict09.columns[[0, 3, 4]], axis = 1)
data_dict09 = data_dict09.iloc[2:598]
data_dict09 = data_dict09.reset_index(drop = True)
data_dict09.columns = ['Variable', 'Label']

data_dict09['Job Category'], data_dict09['Race'], data_dict09['Gender'] = data_dict09['Label'].replace('[0-9.-]', '').str.split('/', 2).str
data_dict09 = data_dict09.drop('Label', axis = 1)

data_dict09['Race'] = data_dict09['Race'].replace('Hawaiia', 'HAWAIIAN')
data_dict09['Race'] = data_dict09['Race'].replace(dict.fromkeys(['Two Races', 'Two More Races'], 'TWO OR MORE RACES'))

lat_df09 = data_dict09[data_dict09['Race'].str.contains('HISPANIC', na = False)]
asn_df09 = data_dict09[data_dict09['Race'].str.contains('ASIAN|Asian', na = False)]

asn_errcor09 = asn_df09[(asn_df09['Race'] != 'ASIAN')].copy()
lat_errcor09 = lat_df09[(lat_df09['Race'] != 'HISPANIC')].copy()

asn_errcor09['Race'] = 'ASIAN'
lat_errcor09['Race'] = 'HISPANIC'

lat_errcor09.loc[122, 'Gender'] = 'FEMALE'

# Updating the data dictionary to be consistent with the preferred variable names for race using original dataframe indices to match.
# Instead of updating each one by row and column, I update the original dataframe with a dataframe comprised of corrected entries.
data_dict09.update(asn_errcor09, overwrite = True)
data_dict09.update(lat_errcor09, overwrite = True)

gen_errcor09 = data_dict09[(data_dict09['Gender'] != 'MALE') & (data_dict09['Gender'] != 'FEMALE') & (data_dict09['Gender'].notnull()) ]
gen_errcor09 = gen_errcor09[~(gen_errcor09['Gender'].str.contains('MINORITY') | gen_errcor09['Gender'].str.contains('PERCENT'))]

data_dict09.loc[144, 'Gender'] = 'MALE'
data_dict09.loc[155, 'Gender'] = 'FEMALE'
data_dict09.loc[354, 'Gender'] = 'MALE(PERCENT)'

#display(gen_errcor09.style.set_caption('Gender Correction'))
#display_side_by_side([asn_errcor09, lat_errcor09], ['Dataframe Error Correction Asian','Dataframe Error Correction Hispanic'])
#display_side_by_side([asn_df09, lat_df09], ['Dataframe Only Asian Race', 'Dataframe Only Hispanic Race'])
display(data_dict09.style.set_caption('2009 Data Dictionary'))
display(df09)

### Question 1.0.

In [ ]:
data_dict09_male = data_dict09[(data_dict09['Gender'] == 'MALE') & (data_dict09['Job Category'] != '10-TOTAL' )]
data_dict09_female = data_dict09[(data_dict09['Gender'] == 'FEMALE') & (data_dict09['Job Category'] != '10-TOTAL' )]

men09 = data_dict09_male['Variable'].to_list()
women09 = data_dict09_female['Variable'].to_list()

df09_ind_men = df09[men09]
df09_ind_women = df09[women09]

df09_ind_men = df09_ind_men.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df09_ind_men["Men's Count"] = df09_ind_men.sum(axis = 1)

df09_ind_women = df09_ind_women.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
df09_ind_women["Women's Count"] = df09_ind_women.sum(axis = 1)

q1_0_09 = pd.DataFrame(index = df09.index)
q1_0_09["Male Count"] = df09_ind_men["Men's Count"]
q1_0_09["Female Count"] = df09_ind_women["Women's Count"]
q1_0_09['Total Count'] = q1_0_09.sum(axis = 1)
q1_0_09['% Men'] = (q1_0_09['Male Count']/q1_0_09['Total Count'])
q1_0_09['% Women'] = (q1_0_09['Female Count']/q1_0_09['Total Count'])
q1_0_09['NAC3 Code Description'] = df09['NAC3_Label']
col_head09 = ['NAC3 Code Description', 'Male Count', 'Female Count', 'Total Count', '% Men', '% Women']
q1_0_09 = q1_0_09[col_head09]

#display(len(q1_0_09.groupby(level = 0)))
display(q1_0_09.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))
#display(q1_0_09.loc[('Minnesota', '541'), ('% Women', 'Total Count')])
#display(df09_ind_men, df09_ind_women)

### Question 1.1.

In [ ]:
# Returns the 10 job categories in the dataset.
job_cat09_list = data_dict09.loc[1:11, 'Job Category']
job_cat09_list = job_cat09_list[job_cat09_list != '10-TOTAL'].to_list()

# Corrects the slight mismatch of job categories.
for i, j in data_dict09.iterrows():
    for k in job_cat09_list:
        if j[1][:2] == k[:2]:
            data_dict09.loc[i, 'Job Category'] = k
            
# List to store the variables corresponding to job category for each race.
job_cat09_male_var = []    
job_cat09_female_var = []    

# Extracting the "variable" element from male data dictionary based on the job category matching across alll races
# then adding to a list.
for k in job_cat09_list:
    for i, j in data_dict09.iterrows():
        if (k == j[1]) and (j[3] == 'MALE'):
            job_cat09_male_var.append(j[0])

for k in job_cat09_list:
    for i, j in data_dict09.iterrows():
        if (k == j[1]) and (j[3] == 'FEMALE'):
            job_cat09_female_var.append(j[0])            
            
# Splits the entire list into 10 arrays of size 7 grouping together the job category across all races.
job_cat09_male_varsplit = np.array_split(job_cat09_male_var, 10)
job_cat09_female_varsplit = np.array_split(job_cat09_female_var, 10)

# Initialize a dictionary where I store all the generated dataframes based on the job category for all races.
job_cat09_male_df_dict = {}
job_cat09_female_df_dict = {}

# Fills the dictionary and replaces all empty cells within each dataframe with an NaN value.
n = 0
for entry in job_cat09_list:
    job_cat09_male_df_dict[entry] = df09[list(job_cat09_male_varsplit[n])]
    #display(entry)
    n += 1
    
n = 0    
for entry in job_cat09_list:
    job_cat09_female_df_dict[entry] = df09[list(job_cat09_female_varsplit[n])]
    #display(entry)
    n += 1
    
q1_1_09_tup_male = []
q1_1_09_tup_female = []

# Returns a list of States from the original dataframe.
df09_idx = sorted(list(set(df09.index.get_level_values(0))))

# Create a tuple with "state, key, sum" then convert into dataframe.
for key, value in job_cat09_male_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df09_idx:
        q1_1_09_tup_male.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

for key, value in job_cat09_female_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    # Creates a new column in every dataframe called "Sum by Industry and Gender" which is the sum of all entries by row per industry.
    value['Sum by Industry and Gender'] = value.sum(axis = 1)  
    #display(key, value)
    for state in df09_idx:
        q1_1_09_tup_female.append(tuple((state, key, value.loc[state]['Sum by Industry and Gender'].sum())))
        #display(state, key, value.loc[state]['Sum by Industry and Gender'].sum())

# Create dataframe based on tuple.
q1_1_09_male = pd.DataFrame(q1_1_09_tup_male, columns = ['STATE_LABEL', 'Job Category', 'Male Count'])
q1_1_09_male = q1_1_09_male.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_09_female = pd.DataFrame(q1_1_09_tup_female, columns = ['STATE_LABEL', 'Job Category', 'Female Count'])
q1_1_09_female = q1_1_09_female.sort_values(by = ['STATE_LABEL', 'Job Category'], ascending = True)
q1_1_09 = pd.merge(q1_1_09_male, q1_1_09_female, on = ['STATE_LABEL', 'Job Category'])     
q1_1_09 = q1_1_09.set_index(['STATE_LABEL', 'Job Category'])
q1_1_09['Total Count'] = q1_1_09.sum(axis = 1)
q1_1_09['% Men'] = q1_1_09['Male Count']/q1_1_09['Total Count']
q1_1_09['% Women'] = q1_1_09['Female Count']/q1_1_09['Total Count']

#display_side_by_side([q1_1_09_male, q1_1_09_female], ['Male Count in Each Job Category', 'Female Count in Each Job Category'])
display(q1_1_09.style.format({'% Men': '{:,.2%}'.format,'% Women': '{:,.2%}'.format}))
#display(job_cat09_male_varsplit)
#display(job_cat09_male_varsplit)
#display(job_gen09_male.style.set_caption('Job Categories by Gender'))

### Question 2.0.

In [ ]:
race_ind09_var = []

for k in race_list:
    for i, j in data_dict09.iterrows():
        if (j[1] != '10-TOTAL') and (j[2] == k) and (j[3] == 'FEMALE' or j[3] == 'MALE'):
            race_ind09_var.append(j[0])
            
race_ind09_varsplit = np.array_split(race_ind09_var, 7)

race_ind09_df_dict = {}

q2_0_09 = pd.DataFrame(index = df09.index)

n = 0
for race in race_list:
    race_ind09_df_dict[race] = df09[list(race_ind09_varsplit[n])]
    n += 1

for key, value in race_ind09_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum by Race'] = value.sum(axis = 1)
    #display(key, value)
    q2_0_09[key] = value['Sum by Race']
    
q2_0_09['NAC3 Code Description'] = df09['NAC3_Label']  
q2_0_09_col_header = ['NAC3 Code Description', 'AM-INDIAN-ALK', 'ASIAN', 'BLACK', 'HAWAIIAN', 'HISPANIC', 'TWO OR MORE RACES', 'WHITE']
q2_0_09 = q2_0_09[q2_0_09_col_header]
q2_0_09 = q2_0_09.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })

q2_0_09['Total Count'] = q2_0_09.sum(axis = 1)
q2_0_09['% American Indian'] = q2_0_09['AM-INDIAN-ALK Count']/q2_0_09['Total Count']
q2_0_09['% Asian'] = q2_0_09['ASIAN Count']/q2_0_09['Total Count']
q2_0_09['% Black'] = q2_0_09['BLACK Count']/q2_0_09['Total Count']
q2_0_09['% Hawaiian'] = q2_0_09['HAWAIIAN Count']/q2_0_09['Total Count']
q2_0_09['% Hispanic'] = q2_0_09['HISPANIC Count']/q2_0_09['Total Count']
q2_0_09['% Two or More Races'] = q2_0_09['TWO OR MORE RACES Count']/q2_0_09['Total Count']
q2_0_09['% White'] = q2_0_09['WHITE Count']/q2_0_09['Total Count']

display(q2_0_09.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))
     


### Question 2.1.

In [ ]:
job_race09_list = []
job_race09_df_dict = {}
job_race09_key = []
job_race09_key_unq = []
job_race09_var = []

df09_idx = sorted(list(set(df09.index.get_level_values(0))))

q2_1_09_tup = []
 
n = 0
for race in race_list:
    for job in job_cat09_list:
        for i, j in data_dict09.iterrows():
            if (j[1] == job) and (j[2] == race) and (j[3] == 'MALE' or j[3] == 'FEMALE'):
                job_race09_list.append(j[0])
                job_race09_key.append((n, race + ', ' + job))
                n += 1

job_race09_arr = np.array_split(job_race09_list, 70)

for arr in job_race09_arr:
    job_race09_var.append(arr.tolist())

for i in job_race09_key:
    if i[0] % 2 == 0:
        job_race09_key_unq.append(i[1])
        
kv = list(zip(job_race09_key_unq, job_race09_var))

for i, j in kv:
    job_race09_df_dict[i] = df09[j]
    
for key, value in job_race09_df_dict.items():
    value = value.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')
    value['Sum of Race by Job Category'] = value.sum(axis = 1)
    #display(key, value)
    for state in df09_idx:
        q2_1_09_tup.append(tuple((state, key.split(',')[0],key.split(',')[1].strip(), value.loc[state]['Sum of Race by Job Category'].sum())))
        
# Use the multilevel index from question 1.1 and column headers to be the race list to set up empty dataframe. Iterate through the q2_1_09_tup list to check what race then use
# q2_1_09.loc[state, job category][race] = q2_1_09_tup[3]
q2_1_09 = pd.DataFrame(index = q1_1_09.index, columns = race_list)  

for tup in q2_1_09_tup:
    q2_1_09.loc[tup[0], tup[2]][tup[1]] = tup[3]
    
q2_1_09 = q2_1_09.rename(columns = {'AM-INDIAN-ALK':'AM-INDIAN-ALK Count', 
                                    'ASIAN':'ASIAN Count', 
                                    'BLACK': 'BLACK Count', 
                                    'HAWAIIAN':'HAWAIIAN Count',
                                    'HISPANIC':'HISPANIC Count',
                                    'TWO OR MORE RACES': 'TWO OR MORE RACES Count',
                                    'WHITE': 'WHITE Count'
                                   })
    
q2_1_09['Total Count'] = q2_1_09.sum(axis = 1)
q2_1_09['% American Indian'] = q2_1_09['AM-INDIAN-ALK Count']/q2_1_09['Total Count']
q2_1_09['% Asian'] = q2_1_09['ASIAN Count']/q2_1_09['Total Count']
q2_1_09['% Black'] = q2_1_09['BLACK Count']/q2_1_09['Total Count']
q2_1_09['% Hawaiian'] = q2_1_09['HAWAIIAN Count']/q2_1_09['Total Count']
q2_1_09['% Hispanic'] = q2_1_09['HISPANIC Count']/q2_1_09['Total Count']
q2_1_09['% Two or More Races'] = q2_1_09['TWO OR MORE RACES Count']/q2_1_09['Total Count']
q2_1_09['% White'] = q2_1_09['WHITE Count']/q2_1_09['Total Count']


display(q2_1_09.style.format({'% American Indian': '{:,.2%}'.format,
                              '% Asian': '{:,.2%}'.format,
                              '% Black': '{:,.2%}'.format,
                              '% Hawaiian': '{:,.2%}'.format,
                              '% Hispanic': '{:,.2%}'.format,
                              '% Two or More Races': '{:,.2%}'.format,
                              '% White': '{:,.2%}'.format,
                             }))


In [ ]:
diff = q2_1_09.loc[("Alaska", "2-PROF")]['Total Count'] - q2_1_09.loc[("Alaska", "2-PROF")]['WHITE Count']

print(diff)

### Question 3.0.

In [ ]:
state_count = 0

print(len(q2_1_09.groupby(level=0)))

t = q2_1_09.index.levels[0]

print(t)

for i in t:
    state_count +=1
    
print(state_count)

In [ ]:
year_list_3_0 = ["2009", "2010","2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018"] # Create year list for second level index.
df_list_3_0 = [q1_1_09, q1_1_10, q1_1_11, q1_1_12, q1_1_13, q1_1_14, q1_1_15, q1_1_16, q1_1_17, q1_1_18] # Dataframes for data extraction.

df_year_3_0 = list(zip(df_list_3_0, year_list_3_0)) # Creates a 2 piece tuple of the dataframe variable and year.
yearSeries_3_0 = pd.Series(year_list_3_0) # Converts a string list to pandas series.

#display(df_year_3_0)

q3_0_id = list(q2_1_09.index.levels[0]) # Extract the state names from question 2.1 dataframe.
tup_list_3_0 = [] # Initialize empty list to create (state, year) tuple.

# Creates a 2 piece tuple that contains first the state name then the year for every state through years 2009 to 2018.
for id in q3_0_id:
    for year in year_list_3_0:
        tup_list_3_0.append([id, year])

# Creates a multilevel index from the tuple list created with labels being 'State' and 'Year'.
q3_0_idx = pd.MultiIndex.from_tuples(tup_list_3_0, names = ["State", "Year"])

# Creating a new dataframe with the multilevel indices and columns for the number of women in the 2-PROF and 3-TECH job category.
q3_0 = pd.DataFrame(index = q3_0_idx, columns = ['Women in 2-PROF', 'Women in 3-TECH'])

# Extracts the count of women in the 2-PROF and 3-TECH job category for all states through 2009 to 2018 and assigns them into the new q3_0 dataframe.
for df, year in df_year_3_0:
    for id in q3_0_id:
        try:
            wprof = df.loc[id, '2-PROF']['Female Count']
            wtech = df.loc[id, '3-TECH']['Female Count']
            q3_0.loc[id, year] = (wprof, wtech)
            
        except:
            q3_0.loc[id, year] = (np.NaN, np.NaN)

display_side_by_side([q3_0.loc['Minnesota'], q3_0.loc['New Mexico']], ['MN', 'NM'])
display(q3_0)


In [ ]:
for i in q3_idx:
    try:
        q1_1_16.loc[i]
    except:
        print("Error")

In [ ]:
a = np.NaN
print(a)

### Question 3.1.

In [ ]:
year_list_3_1 = ["2009", "2010","2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018"] # Create year list for second level index.
df_list_3_1 = [q2_1_09, q2_1_10, q2_1_11, q2_1_12, q2_1_13, q2_1_14, q2_1_15, q2_1_16, q2_1_17, q2_1_18] # Dataframes for data extraction.

df_year_3_1 = list(zip(df_list_3_1, year_list_3_1)) # Creates a 2 piece tuple of the dataframe variable and year.
yearSeries_3_1 = pd.Series(year_list_3_1) # Converts a string list to pandas series.

q3_1_id = list(q2_1_09.index.levels[0]) # Extract the state names question 2.1 dataframe.
tup_list_3_1 = [] # Initialize empty list to create (state, year) tuple.

# Creates a 2 piece tuple that contains first the state name then the year for every state through years 2009 to 2018.
for id in q3_1_id:
    for year in year_list_3_1:
        tup_list_3_1.append([id, year])

# Creates a multilevel index from the tuple list created with labels being 'State' and 'Year'.
q3_1_idx = pd.MultiIndex.from_tuples(tup_list_3_1, names = ["State", "Year"])

# Creating a new dataframe with the multilevel indices and columns for the number of women in the 2-PROF and 3-TECH job category.
q3_1 = pd.DataFrame(index = q3_1_idx, columns = ['Minorities in 2-PROF', 'Minorities in 3-TECH'])

# Extracts the count of women in the 2-PROF and 3-TECH job category for all states through 2009 to 2018 and assigns them into the new q3_0 dataframe.
for df, year in df_year_3_1:
    for id in q3_1_id:
        try:
            minprof = df.loc[id, '2-PROF']['Total Count'] - df.loc[id, '2-PROF']['WHITE Count']
            mintech = df.loc[id, '3-TECH']['Total Count'] - df.loc[id, '3-TECH']['WHITE Count']
            q3_1.loc[id, year] = (minprof, mintech)
            
        except:
            q3_1.loc[id, year] = (np.NaN, np.NaN)

display(q3_1)



### Question 4.0.

In [ ]:
year_list_4_0 = ["2009", "2010","2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018"] # Create year list for second level index.
df_list_4_0 = [q1_1_09, q1_1_10, q1_1_11, q1_1_12, q1_1_13, q1_1_14, q1_1_15, q1_1_16, q1_1_17, q1_1_18] # Dataframes for data extraction.

df_year_4_0 = list(zip(df_list_4_0, year_list_4_0)) # Creates a 2 piece tuple of the dataframe variable and year.
yearSeries_4_0 = pd.Series(year_list_4_0) # Converts a string list to pandas series.

#display(df_year_4_0)

q4_0_id = list(q2_1_09.index.levels[0]) # Extract the state names from question 2.1 dataframe.
tup_list_4_0 = [] # Initialize empty list to create (state, year) tuple.

# Creates a 2 piece tuple that contains first the state name then the year for every state through years 2009 to 2018.
for id in q4_0_id:
    for year in year_list_4_0:
        tup_list_4_0.append([id, year])

# Creates a multilevel index from the tuple list created with labels being 'State' and 'Year'.
q4_0_idx = pd.MultiIndex.from_tuples(tup_list_4_0, names = ["State", "Year"])

# Creating a new dataframe with the multilevel indices and columns for the number of men in the 2-PROF and 3-TECH job category.
q4_0 = pd.DataFrame(index = q4_0_idx, columns = ['Men in 2-PROF', 'Men in 3-TECH'])

# Extracts the count of men in the 2-PROF and 3-TECH job category for all states through 2009 to 2018 and assigns them into the new q4_0 dataframe.
for df, year in df_year_4_0:
    for id in q4_0_id:
        try:
            mprof = df.loc[id, '2-PROF']['Male Count']
            mtech = df.loc[id, '3-TECH']['Male Count']
            q4_0.loc[id, year] = (mprof, mtech)
            
        except:
            q4_0.loc[id, year] = (np.NaN, np.NaN)

#display_side_by_side([q4_0.loc['Minnesota'], q4_0.loc['New Mexico']], ['MN', 'NM'])
display_side_by_side([q4_0.loc['Minnesota'], q3_0.loc['Minnesota'], q3_1.loc['Minnesota']],['Men', 'Women', 'Minorities'])


### Question 4.1.

In [ ]:
year_list_4_1 = ["2009", "2010","2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018"] # Create year list for second level index.
df_list_4_1 = [df09, df10, df11, df12, df13, df14, df15, df16, df17, df18] # Dataframes for data extraction.
data_dict_list = [data_dict09, data_dict10, data_dict11, data_dict12, data_dict13, data_dict14, data_dict15, data_dict16, data_dict17, data_dict18] # Dataframes to extract abbreviated gender-race variable..
data_dict_list_4_1 = [] # Empty list to store copies of the data dictionaries from across all years.
rgen_var_list = []

df_year_4_1 = list(zip(df_list_4_1, year_list_4_1)) # Creates a 2 piece tuple of the dataframe variable and year.
yearSeries_4_1 = pd.Series(year_list_4_1) # Converts a string list to pandas series.

q4_1_id = list(q2_1_09.index.levels[0]) # Extract the state names from question 2.1 dataframe.
tup_list_4_1 = [] # Initialize empty list to create (state, year) tuple.

# Creates a 2 piece tuple that contains first the state name then the year for every state through years 2009 to 2018.
for id in q4_1_id:
    for year in year_list_4_1:
        tup_list_4_1.append([id, year])

# Creates a multilevel index from the tuple list created with labels being 'State' and 'Year'.
q4_1_idx = pd.MultiIndex.from_tuples(tup_list_4_1, names = ["State", "Year"])

# Creating a new dataframe with the multilevel indices and columns for the number of men in the 2-PROF and 3-TECH job category for each Asian and White race.
q4_1 = pd.DataFrame(index = q4_1_idx, columns = ['Asian Men in 2-PROF', 'White Men in 2-PROF', 'Asian Men in 3-TECH', 'White Men in 3-TECH'])

for entry in data_dict_list:
    data_dict_list_4_1.append(entry.copy())

for entry in data_dict_list_4_1:
    temp_var = entry.set_index(['Race', 'Gender', 'Job Category'])
    rgen_var_list.append([temp_var.loc['ASIAN', 'MALE', '2-PROF']['Variable']])
    rgen_var_list.append([temp_var.loc['WHITE', 'MALE', '2-PROF']['Variable']])
    rgen_var_list.append([temp_var.loc['ASIAN', 'MALE', '3-TECH']['Variable']])
    rgen_var_list.append([temp_var.loc['WHITE', 'MALE', '3-TECH']['Variable']])

 
for df, year in df_year_4_1:
    for id in q4_1_id:
        try:
            am2_prof = df.loc[id, '541']['ASIANM2']
            wm2_prof = df.loc[id, '541']['WHM2']
            am3_tech = df.loc[id, '541']['ASIANM3']
            wm2_tech = df.loc[id, '541']['WHM3']
            q4_1.loc[id, year] = (am2_prof, wm2_prof, am3_tech, wm2_tech)
        except:
            q4_1.loc[id, year] =(np.NaN, np.NaN, np.NaN, np.NaN)
            
#test = data_dict_list_4_1[0].set_index(['Race', 'Gender', 'Job Category'])
display(rgen_var_list)
display(q4_1)

In [ ]:
q4_1.loc['Minnesota']